In [1]:
import_libraries = True

if import_libraries:

    import sys
    import os
    import os.path
    from pathlib import Path
    from rdflib import Graph, Namespace, RDF
    from collections import Counter
    import json

    import math
    import pandas as pd
    import numpy as np

    import open3d as o3d

    import laspy
    from sklearn.decomposition import PCA

    from scipy.spatial import ConvexHull
    from scipy.spatial.distance import euclidean

    sys.path.append(r"C:\Users\oscar\IfcOpenShell")

    sys.path.append(r"C:\Users\oscar\anaconda3\envs\cvpr\Lib\site-packages")
    import geomapi
    print(dir(geomapi))  # List available modules inside geomapi

    import geomapi
    import geomapi.tools as tl
    from geomapi.nodes import PointCloudNode
    import geomapi.utils as ut
    from geomapi.utils import geometryutils as gmu
    from geomapi.nodes import PointCloudNode

    import topologicpy 
    from topologicpy.Topology import Topology
    from topologicpy.Vertex import Vertex
    from topologicpy.Edge import Edge
    from topologicpy.Wire import Wire
    from topologicpy.Face import Face
    from topologicpy.Shell import Shell
    from topologicpy.Cell import Cell
    from topologicpy.CellComplex import CellComplex

    from topologicpy.Vector import Vector
    from topologicpy.Plotly import Plotly

    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    from matplotlib import colormaps
    import plotly.graph_objects as go

    import geopy
    from geopy.geocoders import Nominatim
    import math as m
    import pvlib
    import pandas as pd
    from datetime import datetime

    import context_KUL
    import utils_KUL as kul
    import utils_energy as en
        
    import pandas as pd
    import numpy as np
    from shapely import wkt
    from shapely.geometry import Polygon, MultiPolygon
    from shapely.geometry.base import BaseGeometry
    import plotly.graph_objects as go
    import plotly.express as px


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


c:\Users\oscar\anaconda3\envs\cvpr\lib\site-packages\networkx\utils\backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


In [2]:
version_n = 'first_floor_v12_gamma'  
current_directory = Path.cwd()

csv_folder = current_directory / 'outputs' / 'jsons'
csv_file = csv_folder / f'{version_n}.csv'
print(csv_file)


c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\outputs\jsons\first_floor_v12_gamma.csv


In [3]:
folders = True
first_floor_folder = True
extract_topo_info = True

if folders:

    current_directory = Path(os.getcwd()) 
    print(f'Current directory: {current_directory}')

    data_base = Path.cwd().parents[0] / 'data'
    laz_files = list(data_base.glob('**/*.laz'))
    print(f'Found n. {len(laz_files)} in laz folder')

    input_folder_root = Path.cwd().parents[0] / 'outputs' / 'edts'

    print(f"Input root folder: {input_folder_root}")


    names = []

    for name_folder in input_folder_root.iterdir():
        if name_folder.is_dir():  # Ensure it's a directory
            names.append(name_folder.name)
            print(f"Found folder: {name_folder.name}")
            
            json_folder = name_folder / 'json'

            if json_folder.exists():

                json_files = list(json_folder.glob('*.json'))
                print(f"JSON files in '{name_folder.name}' folder:")
                
                for json_file in json_files:
                    filename = json_file.stem
                    extracted_name = filename.split('_walls')[0] if '_walls' in filename else filename
                    print(f"Original filename: {json_file.name}, Extracted name: {extracted_name}")
            else:
                print(f"No 'json' folder found in {name_folder.name}")
            

    print("\nAll files and folders in 'edts' root folder:")
    for file in input_folder_root.glob('*'):
        print(file)

if first_floor_folder:

    from pathlib import Path
    import os

    current_directory = Path(os.getcwd()) 
    print(f'Current directory: {current_directory}')

    data_base = Path.cwd().parents[0] / 'data'
    laz_files = list(data_base.glob('**/*.laz'))
    print(f'Found n. {len(laz_files)}\n     in laz folder {data_base}')

    input_folder_root = Path.cwd().parents[0] / 'outputs' / 'edts'
    print(f"Input root folder: {input_folder_root}")

    target_folder = input_folder_root / 'first_floor'

    if target_folder.exists() and target_folder.is_dir():
        print(f"Working on folder: {target_folder.name}")

        json_folder = target_folder / 'json'

        if json_folder.exists():
            json_files = list(json_folder.glob('*.json'))
            print(f"JSON files in '{target_folder.name}' folder:")

            for json_file in json_files:
                filename = json_file.stem
                extracted_name = filename.split('_walls')[0] if '_walls' in filename else filename
                print(f"Original filename: {json_file.name}, Extracted name: {extracted_name}")
        else:
            print(f"No 'json' folder found in {target_folder.name}")

    else:
        print(f"Folder 'first_floor' does not exist in {input_folder_root}")

if extract_topo_info:

    walls_data_dict = {}

    for name in names:
        
        input_json_walls_file = Path.cwd().parents[0] / 'outputs' / 'edts' / name / 'json' / f'{name}_walls.json'
        print(f"Checking path: {input_json_walls_file}")
        
        if input_json_walls_file.exists():
            print(f"Found file: {input_json_walls_file}")
            try:
                with input_json_walls_file.open() as file:
                    data = json.load(file)
                    print(f"Loaded data for {name}: {data}")

                extracted_data = []

                # Extract walls from the 'first_floor' key
                if "first_floor" in data:
                    walls = data["first_floor"]
                    if isinstance(walls, list):
                        for item in walls:
                            wall_data = {
                                "wall_index": item.get("wall_index"),
                                "center": item.get("center"),
                                "height": item.get("height"),
                                "thickness": item.get("thickness"),
                                "length": item.get("length"),
                                "rotation": item.get("rotation"),
                                "base_center_lines": item.get("base_center_lines"),
                                "top_center_lines": item.get("top_center_lines"),
                                "surface_vertices": item.get("surface_vertices"),
                                "wall_id": item.get("wall_id"),
                                "base_line_start": item.get("base_line", {}).get("start"),
                                "base_line_end": item.get("base_line", {}).get("end")
                            }
                            extracted_data.append(wall_data)
                    else:
                        print(f"Unexpected data structure for walls in {input_json_walls_file}: {walls}")
                else:
                    print(f"Key 'first_floor' not found in {input_json_walls_file}")

                walls_data_dict[name] = extracted_data
                print(f"Data extracted for {name}: {extracted_data}")

            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file {input_json_walls_file}: {e}")
        else:
            print(f"File not found: {input_json_walls_file}")

    if walls_data_dict:
        #print("Final data dictionary:", walls_data_dict)
        print("Final data dictionary found!")
    else:
        print("No data was added to the data dictionary.")

    import json
    from pathlib import Path

    surfaces_data_dict = {}

    for name in names:
        input_json_surfaces_file = Path.cwd().parents[0] / 'outputs' / 'edts' / name / 'json' / f'{name}_walls_centerline_surfaces.json'
        print(f"Checking path: {input_json_surfaces_file}")

        if input_json_surfaces_file.exists():   
            print(f"✅ Found file: {input_json_surfaces_file}")
            try:
                with input_json_surfaces_file.open() as file:
                    data = json.load(file)
                    print(f"📦 Loaded data for {name}")

                extracted_data = []

                # Expecting flat list of centerline face dictionaries
                if isinstance(data, list):
                    for item in data:
                        if item.get("type") == "centerline":
                            wall_data = {
                                "wall_id": item.get("wall_id"),
                                "vertices": item.get("vertices"),  # This is a list of 4 3D points
                            }
                            extracted_data.append(wall_data)
                else:
                    print(f"⚠️ Unexpected data structure in {input_json_surfaces_file}: {type(data)}")

                surfaces_data_dict[name] = extracted_data
                print(f"✅ Surfaces extracted for {name}, total: {len(extracted_data)}")

            except json.JSONDecodeError as e:
                print(f"❌ Error decoding JSON in file {input_json_surfaces_file}: {e}")
        else:
            print(f"❌ File not found: {input_json_surfaces_file}")



    openings_data_dict = {}

    for name in names:
        input_json_openings_file = Path.cwd().parents[0] / 'outputs' / 'edts' / name / 'json' / f'{name}_openings.json'
        print(f"🔍 Checking path: {input_json_openings_file}")

        if input_json_openings_file.exists():
            try:
                with input_json_openings_file.open() as file:
                    data = json.load(file)
                    print(f"✅ Loaded data for {name}")

                extracted_data = []

                if "openings" in data:
                    openings = data["openings"]

                    # ---- DOORS ----
                    if "doors" in openings and isinstance(openings["doors"], list):
                        for door in openings["doors"]:
                            door_data = {
                                "id": door.get("id"),
                                "center": door.get("center"),
                                "bbox_vertices": door.get("bbox_vertices"),
                                "height": door.get("height"),
                                "thickness": door.get("thickness"),
                                "length": door.get("length"),
                                "rotation": door.get("rotation"),
                                "type": "door"
                            }

                            bbox = door_data["bbox_vertices"]
                            center = door_data["center"]
                            length = door_data["length"]
                            thickness = door_data["thickness"]
                            height = door_data["height"]
                            rotation = door_data["rotation"]

                            if not bbox or len(bbox) < 8 or any(len(pt) != 3 for pt in bbox):
                                print(f"⚠️ Fixing malformed DOOR bbox for ID {door_data.get('id')}")

                                try:
                                    # if all(isinstance(x, (int, float)) for x in [length, thickness, height, rotation]) and isinstance(center, list) and len(center) == 3:
                                    if isinstance(center, (list, tuple)) and len(center) == 3 and all(isinstance(c, (int, float)) for c in center):

                                        cx, cy, cz = center
                                        half_l, half_t = length / 2, thickness / 2
                                        box = np.array([
                                            [-half_l, -half_t, 0], [half_l, -half_t, 0],
                                            [half_l, half_t, 0], [-half_l, half_t, 0],
                                            [-half_l, -half_t, height], [half_l, -half_t, height],
                                            [half_l, half_t, height], [-half_l, half_t, height],
                                        ])
                                        rot_matrix = np.array([
                                            [np.cos(rotation), -np.sin(rotation), 0],
                                            [np.sin(rotation),  np.cos(rotation), 0],
                                            [0,                 0,                1]
                                        ])
                                        rotated_box = (rot_matrix @ box.T).T
                                        translated_box = rotated_box + np.array([cx, cy, cz])
                                        door_data["bbox_vertices"] = translated_box.tolist()
                                    else:
                                        print(f"🚫 Skipping malformed door data: {door_data.get('id')}")
                                except Exception as e:
                                    print(f"❌ Failed to fix bbox for door {door_data.get('id')}: {e}")

                            extracted_data.append(door_data)

                    # ---- WINDOWS ----
                    if "windows" in openings and isinstance(openings["windows"], list):
                        for window in openings["windows"]:
                            window_data = {
                                "id": window.get("id"),
                                "center": window.get("center"),
                                "bbox_vertices": window.get("bbox_vertices"),
                                "height": window.get("height"),
                                "thickness": window.get("thickness"),
                                "length": window.get("length"),
                                "rotation": window.get("rotation"),
                                "type": "window"
                            }

                            bbox = window_data["bbox_vertices"]
                            center = window_data["center"]
                            length = window_data["length"]
                            thickness = window_data["thickness"]
                            height = window_data["height"]
                            rotation = window_data["rotation"]

                            if not bbox or len(bbox) < 8 or any(len(pt) != 3 for pt in bbox):
                                print(f"⚠️ Fixing malformed WINDOW bbox for ID {window_data.get('id')}")

                                try:
                                    if all(isinstance(x, (int, float)) for x in [length, thickness, height]) and isinstance(rotation, (int, float)):

                                        cx, cy, cz = center
                                        half_l, half_t = length / 2, thickness / 2
                                        box = np.array([
                                            [-half_l, -half_t, 0], [half_l, -half_t, 0],
                                            [half_l, half_t, 0], [-half_l, half_t, 0],
                                            [-half_l, -half_t, height], [half_l, -half_t, height],
                                            [half_l, half_t, height], [-half_l, half_t, height],
                                        ])
                                        rot_matrix = np.array([
                                            [np.cos(rotation), -np.sin(rotation), 0],
                                            [np.sin(rotation),  np.cos(rotation), 0],
                                            [0,                 0,                1]
                                        ])
                                        rotated_box = (rot_matrix @ box.T).T
                                        translated_box = rotated_box + np.array([cx, cy, cz])
                                        window_data["bbox_vertices"] = translated_box.tolist()
                                    else:
                                        print(f"🚫 Skipping malformed window data: {window_data.get('id')}")
                                except Exception as e:
                                    print(f"❌ Failed to fix bbox for window {window_data.get('id')}: {e}")

                            extracted_data.append(window_data)

                else:
                    print(f"⚠️ Key 'openings' not found in {input_json_openings_file}")

                openings_data_dict[name] = extracted_data
                print(f"📦 Openings data extracted for {name}: {len(extracted_data)} items")

            except Exception as e:
                print(f"❌ Error processing {input_json_openings_file}: {e}")
        else:
            print(f"❌ File not found: {input_json_openings_file}")

name = 'first_floor'

walls_original_bboxes_computation = True

wall_bbox = walls_data_dict[name]  

walls_info = []

for wall in wall_bbox:
    if all(k in wall for k in ['wall_index', 'thickness', 'surface_vertices']):
        wall_data = {
            'wall_index': wall['wall_index'],
            'thickness': wall['thickness'],
            'surface_vertices': wall['surface_vertices']
        }
        walls_info.append(wall_data)

for wall in walls_info:
    print(wall)


Current directory: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script
Found n. 15 in laz folder
Input root folder: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\outputs\edts
Found folder: 01
JSON files in '01' folder:
Original filename: 01_openings.json, Extracted name: 01_openings
Found folder: 01a
JSON files in '01a' folder:
Original filename: 01a_openings.json, Extracted name: 01a_openings
Original filename: 01a_walls.json, Extracted name: 01a
Original filename: 01a_walls_centerline_surfaces.json, Extracted name: 01a
Original filename: 01a_walls_surfaces.json, Extracted name: 01a
Found folder: 01b
JSON files in '01b' folder:
Original filename: 01b_openings.json, Extracted name: 01b_openings
Original filename: 01b_walls.json, Extracted name: 01b
Original filename: 01b_walls_centerline_surfaces.json, Extracted name: 01b
Original filename: 01b_walls_surfaces.json, Extracted name: 01b
Found folder: 08_ShortOffice_01_F1_small_pred
JSON fi

In [4]:
df = pd.read_csv(csv_file)
df_transformed = df.copy()


In [ ]:
# plot_thematic_surface = True

# if plot_thematic_surface:

#     def safe_load_wkt(val):
#         try:
#             if isinstance(val, BaseGeometry):
#                 return val
#             if isinstance(val, str) and val.strip():
#                 geom = wkt.loads(val)
#                 return MultiPolygon([geom]) if isinstance(geom, Polygon) else geom
#         except:
#             return None
#         return None

#     df["geometry"] = df_transformed["geometry_wkt"].apply(safe_load_wkt)

#     def classify_surface(row):
#         if row["thematic_surface"] == "closure":
#             if isinstance(row["parent_id"], str) and row["parent_id"].startswith("room_"):
#                 return "room_closure"
#         return row["thematic_surface"]

#     df["thematic_surface"] = df.apply(classify_surface, axis=1)

#     def refine_wall_labels(row):
#         if row["thematic_surface"] == "partly_wall" and row.get("has_holes") is True:
#             return "external_wall"
#         elif row["thematic_surface"] == "partly_wall":
#             return "internal_wall"
#         return row["thematic_surface"]

#     df["thematic_surface"] = df.apply(refine_wall_labels, axis=1)

#     df = df[df["geometry"].notna()]

#     surface_types = df["thematic_surface"].dropna().unique()
#     colors = px.colors.qualitative.Set2 + px.colors.qualitative.Set3
#     color_map = {s: colors[i % len(colors)] for i, s in enumerate(surface_types)}

#     all_coords = []
#     for geom in df["geometry"]:
#         for poly in geom.geoms:
#             all_coords.extend(list(poly.exterior.coords))

#     all_xyz = np.array(all_coords)
#     x_range = [all_xyz[:, 0].min(), all_xyz[:, 0].max()]
#     y_range = [all_xyz[:, 1].min(), all_xyz[:, 1].max()]
#     z_range = [all_xyz[:, 2].min(), all_xyz[:, 2].max()]

#     aspect = {
#         "x": max(x_range[1] - x_range[0], 1e-6),
#         "y": max(y_range[1] - y_range[0], 1e-6),
#         "z": max(z_range[1] - z_range[0], 1e-6)
#     }
#     max_range = max(aspect.values())
#     aspect_ratio = {axis: round(aspect[axis] / max_range, 4) for axis in aspect}

#     fig = go.Figure()

#     for _, row in df.iterrows():
#         color = color_map.get(row["thematic_surface"], "gray")
#         for poly in row["geometry"].geoms:
#             coords = np.array(poly.exterior.coords)
#             hover_text = f"""
#             Type: {row['thematic_surface']}<br>
#             Element ID: {row['element_id']}<br>
#             Parent: {row['parent_id']}<br>
#             Has Holes: {row['has_holes']}
#             """
#             fig.add_trace(go.Scatter3d(
#                 x=coords[:, 0], y=coords[:, 1], z=coords[:, 2],
#                 mode='lines',
#                 line=dict(color=color, width=4.5),
#                 name=row["thematic_surface"],
#                 showlegend=False,
#                 hoverinfo="text",
#                 text=hover_text
#             ))

#     for thematic, color in color_map.items():
#         fig.add_trace(go.Scatter3d(
#             x=[None], y=[None], z=[None],
#             mode='markers',
#             marker=dict(size=6, color=color),
#             name=thematic
#         ))

#     fig.update_layout(
#         scene=dict(
#             xaxis=dict(title='X', range=x_range, backgroundcolor='white', gridcolor='lightgray', zerolinecolor='gray'),
#             yaxis=dict(title='Y', range=y_range, backgroundcolor='white', gridcolor='lightgray', zerolinecolor='gray'),
#             zaxis=dict(title='Z', range=z_range, backgroundcolor='white', gridcolor='lightgray', zerolinecolor='gray'),
#             aspectmode='manual',
#             aspectratio=aspect_ratio,
#             bgcolor='white'
#         ),
#         paper_bgcolor='white',
#         plot_bgcolor='white',
#         title="3D Building Model — Colored by Thematic Surface",
#         legend=dict(title="Thematic Surface", itemsizing="constant"),
#         margin=dict(l=10, r=10, b=10, t=40),
#         width=1800,
#         height=900
#     )

#     fig.write_html("3d_thematic_model.html", auto_open=True)


In [9]:
assign_thickness_step_01 = True

if assign_thickness_step_01:

    import pandas as pd
    import numpy as np
    from shapely.geometry import Polygon, MultiPolygon
    from shapely.geometry.base import BaseGeometry
    from shapely.wkt import loads as load_wkt

    df = df_transformed.copy()

    def safe_load_wkt(val):
        try:
            if isinstance(val, BaseGeometry):
                return val
            if isinstance(val, str) and val.strip():
                geom = load_wkt(val)
                return MultiPolygon([geom]) if isinstance(geom, Polygon) else geom
        except Exception as e:
            return None
        return None

    df["geometry"] = df["geometry_wkt"].apply(safe_load_wkt)

    def classify_surface(row):
        if row["thematic_surface"] == "closure" and isinstance(row["parent_id"], str) and row["parent_id"].startswith("room_"):
            return "room_closure"
        return row["thematic_surface"]

    df["thematic_surface"] = df.apply(classify_surface, axis=1)

    def refine_wall_labels(row):
        label = row["thematic_surface"]
        has_holes = bool(row.get("has_holes"))

        if label in ["partly_wall", "partly_wall_with_holes"]:
            return "external_wall" if has_holes else "internal_wall"
        elif label in ["external", "external_with_holes"]:
            return "external_wall" if has_holes else "external_wall"  # both map to external_wall

        return label

    df["thematic_surface"] = df.apply(refine_wall_labels, axis=1)
    df = df[df["geometry"].notna()]

    wall_types = ["external_wall", "internal_wall"]
    df["wall_type_detailed"] = df.apply(
        lambda row: f"{row['thematic_surface']}_with_holes"
        if bool(row.get("has_holes")) and row["thematic_surface"] in wall_types
        else row["thematic_surface"],
        axis=1
    )

    df_walls = df[df["wall_type_detailed"].isin([
        "external_wall", "external_with_holes",
        "internal_wall", "partly_wall_with_holes"
    ])]

    thematic_walls = []

    for _, row in df_walls.iterrows():
        for poly in row["geometry"].geoms:
            coords = list(poly.exterior.coords)
            thematic_walls.append({
                "element_id": row["element_id"],
                "parent_id": row["parent_id"],
                "wall_type": row["wall_type_detailed"],
                "has_holes": row["has_holes"],
                "surface_vertices": coords
            })

    df_thematic_walls = pd.DataFrame(thematic_walls)

    print("✅ Wall types found:")
    print(df_thematic_walls["wall_type"].value_counts())

    # Define additional surface types to extract
    surface_types_to_extract = [
        "external_wall", "internal_wall", "external_with_holes", "partly_wall_with_holes", "closure",
        "ceiling", "floor", "roof"  # add any additional thematic surfaces as needed
    ]

    # Filter the main dataframe to include the target surfaces
    df_surfaces = df[df["thematic_surface"].isin(surface_types_to_extract)]

    # Container for all extracted surfaces
    thematic_surfaces = []

    for _, row in df_surfaces.iterrows():
        for poly in row["geometry"].geoms:
            coords = list(poly.exterior.coords)
            thematic_surfaces.append({
                "element_id": row["element_id"],
                "parent_id": row["parent_id"],
                "thematic_surface": row["thematic_surface"],
                "wall_type": row.get("wall_type_detailed", None),  # wall type if available
                "has_holes": row.get("has_holes", False),
                "surface_vertices": coords
            })

    # Create DataFrame with all extracted thematic surfaces
    df_thematic_surfaces = pd.DataFrame(thematic_surfaces)

    # Optional: print counts
    print("✅ Extracted surface types:")
    print(df_thematic_surfaces["thematic_surface"].value_counts())



✅ Wall types found:
wall_type
external_wall    20
Name: count, dtype: int64
✅ Extracted surface types:
thematic_surface
external_wall    21
ceiling          21
floor            21
closure           3
Name: count, dtype: int64


In [11]:
assign_thickness_step_02 = True

if assign_thickness_step_02:

    def get_bbox(surface_vertices):
        vertices = np.array(surface_vertices)
        min_xyz = vertices.min(axis=0)
        max_xyz = vertices.max(axis=0)
        return min_xyz, max_xyz

    def get_centroid(coords):
        coords = np.array(coords)
        return coords.mean(axis=0)

    def find_thickness(centroid, wall_bboxes):
        for bbox in wall_bboxes:
            if np.all(centroid >= bbox["min_xyz"]) and np.all(centroid <= bbox["max_xyz"]):
                return bbox["thickness"]
        return np.nan 
    
    def assign_standard_thickness(row):

        wall_type = str(row.get("wall_type", "")).strip().lower()
      
        if pd.isna(row["matched_thickness"]):
            if wall_type == "external":
                return 0.35
            elif wall_type == "partly":
                return 0.125
      
        return row["matched_thickness"]
    
    def assign_standard_thickness(row):
        surface_type = str(row.get("thematic_surface", "")).strip().lower()
        wall_type = str(row.get("wall_type", "")).strip().lower()

        if pd.isna(row["matched_thickness"]):
      
            if wall_type == "external":
                return 0.35
            elif wall_type == "partly":
                return 0.125
            elif surface_type == "ceiling":
                return 0.30  # default ceiling thickness
            elif surface_type == "floor":
                return 0.35  # default floor thickness
      
        return row["matched_thickness"]

    # round_thickness_value = 0.025

    # wall_bboxes = []

    # for wall in walls_info:
    #     min_xyz, max_xyz = get_bbox(wall["surface_vertices"])
    #     wall_bboxes.append({
    #         "wall_index": wall["wall_index"],
    #         "thickness": wall["thickness"],
    #         "min_xyz": min_xyz,
    #         "max_xyz": max_xyz
    #     })

    # df_thematic_walls["centroid"] = df_thematic_walls["surface_vertices"].apply(get_centroid)

    # df_thematic_walls["matched_thickness"] = df_thematic_walls["centroid"].apply(
    #     lambda c: find_thickness(c, wall_bboxes)
    # )
    
    # df_thematic_walls["matched_thickness"] = df_thematic_walls.apply(assign_standard_thickness, axis=1)

    # df_thematic_walls["thickness_rounded"] = df_thematic_walls["matched_thickness"].apply(
    #     lambda t: round(t / round_thickness_value) * round_thickness_value if pd.notna(t) else np.nan
    # )

    # df_thematic_walls.head(50)

    round_thickness_value = 0.025

    # Step 1: Build wall bounding boxes
    wall_bboxes = []
    for wall in walls_info:
        min_xyz, max_xyz = get_bbox(wall["surface_vertices"])
        wall_bboxes.append({
            "wall_index": wall["wall_index"],
            "thickness": wall["thickness"],
            "min_xyz": min_xyz,
            "max_xyz": max_xyz
        })

    # Step 2: Assign centroids
    df_thematic_surfaces["centroid"] = df_thematic_surfaces["surface_vertices"].apply(get_centroid)

    # Step 3: Match wall thickness using geometry
    df_thematic_surfaces["matched_thickness"] = df_thematic_surfaces["centroid"].apply(
        lambda c: find_thickness(c, wall_bboxes)
    )

    # Step 4: Assign defaults based on surface type
    def assign_standard_thickness(row):
        surface_type = str(row.get("thematic_surface", "")).lower()
        wall_type = str(row.get("wall_type", "")).lower()

        if pd.isna(row["matched_thickness"]):
            if wall_type == "external":
                return 0.35
            elif wall_type == "partly":
                return 0.125
            elif surface_type == "ceiling":
                return 0.30
            elif surface_type == "floor":
                return 0.35
        return row["matched_thickness"]

    df_thematic_surfaces["matched_thickness"] = df_thematic_surfaces.apply(assign_standard_thickness, axis=1)

    # Step 5: Apply rounding
    df_thematic_surfaces["thickness_rounded"] = df_thematic_surfaces["matched_thickness"].apply(
        lambda t: round(t / round_thickness_value) * round_thickness_value if pd.notna(t) else np.nan
)


In [12]:
df_thematic_surfaces["matched_thickness"] = df_thematic_surfaces["matched_thickness"].fillna(0.150)  # or any default
df_thematic_surfaces.head(50)


,element_id,parent_id,thematic_surface,wall_type,has_holes,surface_vertices,centroid,matched_thickness,thickness_rounded
0,3,1,external_wall,external_wall,False,"[(-39.119, 29.884, 6.657), (-31.666, 29.983, 6...","[-36.1378, 29.9236, 5.215]",0.644637,0.650
1,5,1,external_wall,external_wall,False,"[(-31.666, 29.983, 3.052), (-31.666, 29.983, 6...","[-31.6048, 27.1762, 4.494]",0.343848,0.350
2,6,1,external_wall,external_wall,False,"[(-31.513, 22.966, 3.052), (-31.513, 22.966, 6...","[-31.5086, 22.7748, 4.494]",0.150000,NaN
3,10,1,ceiling,ceiling,False,"[(-39.014, 22.381, 6.657), (-31.502, 22.488, 6...","[-35.30466666666667, 25.013833333333338, 6.656...",0.300000,0.300
4,9,1,floor,floor,False,"[(-39.014, 22.381, 3.052), (-39.119, 29.884, 3...","[-35.30466666666667, 25.013833333333334, 3.052]",0.350000,0.350
5,16,11,external_wall,external_wall,False,"[(-46.396, 29.788, 6.657), (-39.119, 29.884, 6...","[-43.4852, 29.8264, 5.215]",0.644637,0.650
6,19,11,ceiling,ceiling,False,"[(-39.119, 29.884, 6.657), (-46.396, 29.788, 6...","[-41.9686, 26.842999999999996, 6.656999999999999]",0.300000,0.300
7,18,11,floor,floor,False,"[(-39.119, 29.884, 3.052), (-39.014, 22.381, 3...","[-41.968599999999995, 26.843, 3.052]",0.350000,0.350
8,41,20,external_wall,external_wall,False,"[(-53.716, 13.593, 6.657), (-55.572, 13.498, 6...","[-54.458400000000005, 13.555000000000001, 5.215]",0.353900,0.350
9,53,20,ceiling,ceiling,False,"[(-46.195, 22.278, 6.657), (-49.506, 22.23, 6....","[-43.218653846153835, 17.881192307692306, 6.65...",0.300000,0.300


VISUAL CHECK

In [18]:
overlapping_visual_check = True

if overlapping_visual_check:

    import plotly.graph_objects as go
    import numpy as np

    fig = go.Figure()

    def draw_bbox(fig, min_xyz, max_xyz, color="red", name="BBox"):
        # 8 corners of the box
        x0, y0, z0 = min_xyz
        x1, y1, z1 = max_xyz

        corners = np.array([
            [x0, y0, z0],
            [x1, y0, z0],
            [x1, y1, z0],
            [x0, y1, z0],
            [x0, y0, z1],
            [x1, y0, z1],
            [x1, y1, z1],
            [x0, y1, z1]
        ])

        # Pairs of indices for edges
        edges = [
            (0, 1), (1, 2), (2, 3), (3, 0),
            (4, 5), (5, 6), (6, 7), (7, 4),
            (0, 4), (1, 5), (2, 6), (3, 7)
        ]

        for i, j in edges:
            fig.add_trace(go.Scatter3d(
                x=[corners[i][0], corners[j][0]],
                y=[corners[i][1], corners[j][1]],
                z=[corners[i][2], corners[j][2]],
                mode="lines",
                line=dict(color=color, width=2),
                showlegend=False,
                name=name
            ))

    for _, row in df_thematic_walls.iterrows():
        coords = np.array(row["surface_vertices"])
        fig.add_trace(go.Scatter3d(
            x=coords[:, 0], y=coords[:, 1], z=coords[:, 2],
            mode="lines",
            line=dict(color="blue", width=3),
            name=f"Thematic: {row['wall_type']}",
            hoverinfo="text",
            text=f"ID: {row['element_id']}<br>Type: {row['wall_type']}<br>Th: {row.get('thickness_rounded', '')}"
        ))

    for wall in walls_info:
        min_xyz, max_xyz = get_bbox(wall["surface_vertices"])
        draw_bbox(fig, min_xyz, max_xyz,
                color="red",
                name=f"BBox Wall {wall['wall_index']}")


    # === Layout ===
    fig.update_layout(
        title="Overlap of Thematic Surfaces and Wall Bounding Boxes",
        scene=dict(
            xaxis_title="X", yaxis_title="Y", zaxis_title="Z",
            aspectmode="data",
            bgcolor="white"
        ),
        legend=dict(itemsizing="constant"),
        margin=dict(l=10, r=10, t=40, b=10),
        width=1400, height=800
    )

    fig.show()


c:\Users\oscar\anaconda3\envs\cvpr\lib\site-packages\kaleido\__init__.py:14: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




ASSIGN MATERIALS

In [19]:
import_materials_and_constructions = True

if import_materials_and_constructions:

    import uuid
    import ast
    from ast import literal_eval

    df = pd.read_csv(csv_file)

    current_directory = os.getcwd()
    json_materials = os.path.join(current_directory, 'IDL_files', 'materials_constructions', 'materials.json')  
    json_constructions = os.path.join(current_directory, 'IDL_files', 'materials_constructions', 'constructions.json')

    with open(json_materials, 'r') as f:
        materials_library = json.load(f)
    
    with open(json_constructions, 'r') as m:
        constructions_library =  json.load(m)

    df_materials = pd.DataFrame(materials_library)
    df_constructions = pd.DataFrame(constructions_library)


    df_constructions = df_constructions.copy()
    df_constructions['construction_uuid'] = [str(uuid.uuid4()) for _ in range(len(df_constructions))]

# df_materials.head()
df_constructions.head(80)

import_materials_and_constructions = True

if import_materials_and_constructions:

    current_directory = os.getcwd()
    json_materials_path = os.path.join(current_directory, 'IDL_files', 'materials_constructions', 'materials.json')
    json_constructions_path = os.path.join(current_directory, 'IDL_files', 'materials_constructions', 'constructions.json')

    with open(json_materials_path, 'r') as l:
        materials_library = json.load(l)

    with open(json_constructions_path, 'r') as m:
        constructions_library = json.load(m)

    df_material = pd.DataFrame(materials_library)
    df_constructions = pd.DataFrame(constructions_library)
    df_constructions['construction_uuid'] = [str(uuid.uuid4()) for _ in range(len(df_constructions))]


In [20]:
section_to_define_csv_materialconstructions = True 

if section_to_define_csv_materialconstructions:

    import pandas as pd
    import os
    import json
    import uuid

    # Assume df_material is already loaded
    df = df_material.copy()

    # --- Add material metadata ---
    df['library_id'] = 'mat_lib_id_01'
    df['mat_lib_id'] = 'material_id_' + df['material_id'].astype(str)
    df['class'] = df['name'].apply(lambda x: 'gas_material' if 'argon' in x.lower() else 'solid_material')
    df['gml_id'] = df['uuid']
    df['description'] = df['name'].apply(
        lambda x: f"Gas fill: {x.replace('_', ' ')}." if 'argon' in x.lower() else f"A sample of {x.replace('_', ' ')}."
    )
    df['thermal_conductivity'] = df['lambda_W_per_mK']
    df['thermal_conductivity_uom'] = np.where(df['thermal_conductivity'].notnull(),'W/(m.K)', '')

    df['density'] = df['density_kg_per_m3']
    df['density_uom'] = np.where(df['density'].notna(), 'kg/m3', '')

    df['spec_heat_cap'] = df['spec_heat_cap'] if 'spec_heat_cap' in df else ''
    df['spec_heat_cap'] = df['spec_heat_cap'].fillna('')
    df['spec_heat_cap_uom'] = np.where(df['spec_heat_cap'] != '', 'J/(kg.K)', '')

    df['is_ventilated'] = df['class'].apply(lambda x: True if x == 'gas_material' else False)

    df['r_value'] = df['r_value'] if 'r_value' in df else ''
    df['r_value'] = df['r_value'].fillna('')
    df['r_value_uom'] = np.where(df['r_value'] != '', '(m2.K)/W', '')

    new_columns = [
        'library_id', 'mat_lib_id', 'class', 'gml_id', 'name', 'description',
        'thermal_conductivity', 'thermal_conductivity_uom', 'density', 'density_uom',
        'spec_heat_cap', 'spec_heat_cap_uom', 'is_ventilated', 'r_value', 'r_value_uom'
    ]
    formatted_df = df[new_columns]

    # Normalize names for lookup
    df['name_norm'] = df['name'].str.strip().str.lower()
    name_to_matid = dict(zip(df['name_norm'], df['mat_lib_id']))
    uuid_to_matid = dict(zip(df['gml_id'], df['material_id']))

    # Load constructions
    with open(r"C:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\IDL_files\materials_constructions\constructions.json", "r") as file:
        constructions = json.load(file)

    # Prepare paths
    output_materials_path = r"C:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\IDL_files\materials_constructions"
    materials_csv_path = os.path.join(output_materials_path, "Materials.csv")
    layers_csv_path = os.path.join(output_materials_path, "Layers.csv")
    library_csv_path = os.path.join(output_materials_path, "LayeredConstructions.csv")

    # Export materials
    formatted_df.to_csv(materials_csv_path, index=False)

    # --- Construct layers and libraries ---
    layer_rows = []
    library_rows = []

    for idx, constr in enumerate(constructions, start=1):
        constr_id = f"LayeredConstr_id_{idx:04}"
        library_id = f"LayConst_lib_id_01"

        # Compute total R-value
        r_total = 0.0
        for layer in constr["layers"]:
            material_name = layer["material"].strip().lower()
            thickness = layer["thickness"]
            mat_row = df[df['name_norm'] == material_name]
            if not mat_row.empty:
                lam = mat_row.iloc[0]['lambda_W_per_mK']
                if pd.notna(lam) and lam > 0:
                    r_total += thickness / lam

        uvalue = round(1 / r_total, 3) if r_total > 0 else ""
        uvalue_uom = "W/(K.m2)" if uvalue else ""

        # Add layer rows
        for layer in constr["layers"]:
            material_name_raw = layer["material"]
            material_name = material_name_raw.strip().lower()
            material_uuid = layer.get("material_uuid")

            if material_uuid and material_uuid in uuid_to_matid:
                material_id = f"material_id_{uuid_to_matid[material_uuid]}"
            else:
                material_id = name_to_matid.get(material_name, "material_id_NULL")

            layer_row = {
                "parent_id": constr_id,
                "gmlid": f"Layer_id_{uuid.uuid4()}",
                "name": material_name,
                "description": f"{material_name} layer, {round(layer['thickness'] * 1000)} mm thick",
                "thickness": round(layer["thickness"] * 1000, 2),
                "thickness_uom": "mm",
                "material_id": material_id
            }
            layer_rows.append(layer_row)

        print(f"→ Checking construction: {constr['name']}")

        matched_glass = ""
        glazing_ratio = constr.get("glazing_ratio", "")
        glazing_ratio_uom = " " if glazing_ratio not in ("", None) else ""

        if not glazing_ratio:
            for layer in constr["layers"]:
                mat_name = layer["material"].strip().lower()
                print(f"   Layer material: {mat_name}")  
                if "glass_triple" in mat_name:
                    glazing_ratio = 0.5
                    matched_glass = "glass_triple"
                    uvalue = 1.2
                    break
                elif "glass_double" in mat_name:
                    glazing_ratio = 0.65
                    matched_glass = "glass_double"
                    uvalue = 2.8
                    break
                elif mat_name == "glass":
                    glazing_ratio = 0.6
                    matched_glass = "glass"
                    uvalue = 3.6
                    break

            glazing_ratio_uom = "" if glazing_ratio != "" else ""

        # ✅ Move transparency_fraction assignment here
        transparency_fraction = ""
        if matched_glass == "glass_triple":
            transparency_fraction = 0.65
        elif matched_glass == "glass_double":
            transparency_fraction = 0.75
        elif matched_glass == "glass":
            transparency_fraction = 0.85

        # Rename if needed
        if "window" not in constr["name"].lower() and matched_glass:
            constr["name"] = f"window_{matched_glass}"
            print(f"🔧 Renamed to: {constr['name']}")

        # Check if it's an insulated wall
        is_insulated = any('insulation_eps' in layer['material'].lower() for layer in constr["layers"])
        description = f"{constr['wall_type']} with total thickness {round(constr['match_thickness'] * 1000)} mm"
        if is_insulated:
            description += " (insulated wall)"

        library_row = {
            "library_id": library_id,
            "class": "LayeredConstruction",
            "gmlid": constr_id,
            "name": constr["name"],
            "description": description,
            "librarycode": "",
            "librarycode_codespace": "",
            "glazing_ratio": glazing_ratio,
            "glazing_ratio_uom": glazing_ratio_uom,
            "uvalue": uvalue,
            "uvalue_uom": uvalue_uom,
            "transparency_fraction": transparency_fraction,
            "emissivity_fraction": "",
            "emissivity_side": "",
            "reflectance_fraction": "",
            "reflectance_wawelength_range": "",
            "transmittance_fraction": "",
            "transmittance_side": "",
            "transmittance_wawelength_range": "",
            "layered_construction_id": ""
        }
        library_rows.append(library_row)
    
    # --- Export ---
    layers_df = pd.DataFrame(layer_rows)
    library_df = pd.DataFrame(library_rows)

    # Remap gmlid to layered_construction_id
    library_df["layered_construction_id"] = library_df["gmlid"]

    # Map: construction_id → first matching layer gmlid
    first_layer_gmlids = (
        layers_df.groupby("parent_id")["gmlid"]
        .first()
        .to_dict()
    )

    # Add cross-reference by matching material name or uuid
    df['name_norm'] = df['name'].str.strip().str.lower()
    name_to_uuid = dict(zip(df['name_norm'], df['gml_id']))

    # Try to get uuid or name from matched material
    def resolve_material_reference(row):
        name_norm = row['name'].strip().lower()
        mat_id = row['material_id']
        
        matched_uuid = name_to_uuid.get(name_norm, "")
        matched_entry = df[df['mat_lib_id'] == mat_id]
        
        if not matched_entry.empty:
            return f"✔ Matched by ID ({matched_entry.iloc[0]['name']})"
        elif matched_uuid:
            return f"material_id_{matched_uuid}"
        else:
            return "❌ No match"

    # Add a new reference column
    layers_df['material_reference'] = layers_df.apply(resolve_material_reference, axis=1)

    layers_df.insert(0, 'library_id', 'LayConstLib_id_02')
    library_df["gmlid"] = library_df["layered_construction_id"].map(first_layer_gmlids)
    
    layers_df.to_csv(layers_csv_path, index=False)
    # LayeredConstructions
    library_df.to_csv(library_csv_path, index=False)



→ Checking construction: external_wall_brick_10cm
   Layer material: plaster_external
   Layer material: bricks_full
   Layer material: mortar
→ Checking construction: external_wall_brick_12cm
   Layer material: plaster_external
   Layer material: bricks_full
   Layer material: mortar
→ Checking construction: external_wall_brick_15cm
   Layer material: plaster_external
   Layer material: bricks_full
   Layer material: mortar
→ Checking construction: external_wall_brick_17cm
   Layer material: plaster_external
   Layer material: bricks_full
   Layer material: mortar
→ Checking construction: external_wall_brick_20cm
   Layer material: plaster_external
   Layer material: bricks_full
   Layer material: mortar
→ Checking construction: external_wall_brick_25cm
   Layer material: plaster_external
   Layer material: bricks_full
   Layer material: mortar
→ Checking construction: external_wall_brick_30cm
   Layer material: plaster_external
   Layer material: bricks_full
   Layer material: mortar

In [21]:
# CONSTRUCTIONS_LIBRARY

pd.set_option('display.max_colwidth', 600)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
# formatted_df.head(60)
library_df.head(80)

from IPython.display import display, HTML
display(HTML(library_df.head(160).to_html()))


,library_id,class,gmlid,name,description,librarycode,librarycode_codespace,glazing_ratio,glazing_ratio_uom,uvalue,uvalue_uom,transparency_fraction,emissivity_fraction,emissivity_side,reflectance_fraction,reflectance_wawelength_range,transmittance_fraction,transmittance_side,transmittance_wawelength_range,layered_construction_id
0,LayConst_lib_id_01,LayeredConstruction,Layer_id_a48d94d6-2f7d-4299-998e-d91b72d00fd9,external_wall_brick_10cm,external_wall with total thickness 125 mm,,,,,5.999,W/(K.m2),,,,,,,,,LayeredConstr_id_0001
1,LayConst_lib_id_01,LayeredConstruction,Layer_id_167d271a-ded9-4b2c-8742-3c795cf062bc,external_wall_brick_12cm,external_wall with total thickness 150 mm,,,,,4.965,W/(K.m2),,,,,,,,,LayeredConstr_id_0002
2,LayConst_lib_id_01,LayeredConstruction,Layer_id_c07d0f0d-004d-4260-b55f-506b37b9ee89,external_wall_brick_15cm,external_wall with total thickness 175 mm,,,,,4.235,W/(K.m2),,,,,,,,,LayeredConstr_id_0003
3,LayConst_lib_id_01,LayeredConstruction,Layer_id_43d4fd91-b8e6-49ca-a947-4b4ca5f757e6,external_wall_brick_17cm,external_wall with total thickness 200 mm,,,,,3.692,W/(K.m2),,,,,,,,,LayeredConstr_id_0004
4,LayConst_lib_id_01,LayeredConstruction,Layer_id_9da8efb6-4050-4b7a-870e-3c3a0f4722de,external_wall_brick_20cm,external_wall with total thickness 225 mm,,,,,3.272,W/(K.m2),,,,,,,,,LayeredConstr_id_0005
5,LayConst_lib_id_01,LayeredConstruction,Layer_id_c4ff362d-204e-407f-a97b-763f5b7d8e3a,external_wall_brick_25cm,external_wall with total thickness 275 mm,,,,,2.666,W/(K.m2),,,,,,,,,LayeredConstr_id_0006
6,LayConst_lib_id_01,LayeredConstruction,Layer_id_e5828bde-82f9-4630-8053-0c188213c11c,external_wall_brick_30cm,external_wall with total thickness 325 mm,,,,,2.250,W/(K.m2),,,,,,,,,LayeredConstr_id_0007
7,LayConst_lib_id_01,LayeredConstruction,Layer_id_083f5ce6-d8bd-4ce4-9318-20464967bb04,internal_wall_brick_10cm,internal_wall with total thickness 125 mm,,,,,5.999,W/(K.m2),,,,,,,,,LayeredConstr_id_0008
8,LayConst_lib_id_01,LayeredConstruction,Layer_id_96c9fe88-beac-4088-9b60-5d84824614a6,internal_wall_brick_12cm,internal_wall with total thickness 150 mm,,,,,4.965,W/(K.m2),,,,,,,,,LayeredConstr_id_0009
9,LayConst_lib_id_01,LayeredConstruction,Layer_id_4ef149c0-d917-4bc8-a448-90161852e287,internal_wall_brick_15cm,internal_wall with total thickness 175 mm,,,,,4.235,W/(K.m2),,,,,,,,,LayeredConstr_id_0010


In [13]:
# CONSTRUCTIONS_LIBRARY

pd.set_option('display.max_colwidth', 600)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
# formatted_df.head(60)
layers_df.head(80)

from IPython.display import display, HTML
display(HTML(layers_df.head(500).to_html()))

# Layer_id_0b88843d-9890-4fae-a3ec-a8a54caa191c


,library_id,parent_id,gmlid,name,description,thickness,thickness_uom,material_id,material_reference
0,LayConstLib_id_02,LayeredConstr_id_0001,Layer_id_2c26b5e1-0f22-4017-88e8-6b315331416a,plaster_external,"plaster_external layer, 12 mm thick",12.5,mm,material_id_113,✔ Matched by ID (plaster_external)
1,LayConstLib_id_02,LayeredConstr_id_0001,Layer_id_681a56a1-8798-4475-816a-3e7f7acb43a3,bricks_full,"bricks_full layer, 100 mm thick",100.0,mm,material_id_102,✔ Matched by ID (bricks_full)
2,LayConstLib_id_02,LayeredConstr_id_0001,Layer_id_330c0bf7-5faa-4932-925d-93cce187e618,mortar,"mortar layer, 12 mm thick",12.5,mm,material_id_202,✔ Matched by ID (mortar)
3,LayConstLib_id_02,LayeredConstr_id_0002,Layer_id_307c358b-b400-4025-b85e-a1ce4c3dc03b,plaster_external,"plaster_external layer, 12 mm thick",12.5,mm,material_id_113,✔ Matched by ID (plaster_external)
4,LayConstLib_id_02,LayeredConstr_id_0002,Layer_id_2efb60c1-922e-42c7-b038-1a14d59e7a91,bricks_full,"bricks_full layer, 125 mm thick",125.0,mm,material_id_102,✔ Matched by ID (bricks_full)
5,LayConstLib_id_02,LayeredConstr_id_0002,Layer_id_232b8bda-6f4e-46cb-bb47-3d54ab9eb74e,mortar,"mortar layer, 12 mm thick",12.5,mm,material_id_202,✔ Matched by ID (mortar)
6,LayConstLib_id_02,LayeredConstr_id_0003,Layer_id_944c5631-35e3-4375-8afe-942aacc1f0f5,plaster_external,"plaster_external layer, 12 mm thick",12.5,mm,material_id_113,✔ Matched by ID (plaster_external)
7,LayConstLib_id_02,LayeredConstr_id_0003,Layer_id_0ff917c3-73bd-41a6-adcb-c5fdc79646c5,bricks_full,"bricks_full layer, 150 mm thick",150.0,mm,material_id_102,✔ Matched by ID (bricks_full)
8,LayConstLib_id_02,LayeredConstr_id_0003,Layer_id_b65143be-48ed-40ca-b07e-5821edfc3512,mortar,"mortar layer, 12 mm thick",12.5,mm,material_id_202,✔ Matched by ID (mortar)
9,LayConstLib_id_02,LayeredConstr_id_0004,Layer_id_016dda44-bda1-46cd-bf7f-002a2ee1b392,plaster_external,"plaster_external layer, 12 mm thick",12.5,mm,material_id_113,✔ Matched by ID (plaster_external)


In [14]:
# MATERIALS

pd.set_option('display.max_colwidth', 600)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
# formatted_df.head(60)
formatted_df.head(80)

from IPython.display import display, HTML
display(HTML(formatted_df.head(100).to_html()))


,library_id,mat_lib_id,class,gml_id,name,description,thermal_conductivity,thermal_conductivity_uom,density,density_uom,spec_heat_cap,spec_heat_cap_uom,is_ventilated,r_value,r_value_uom
0,mat_lib_id_01,material_id_101,solid_material,b0a6a2d1-1001-4ff6-a7f9-1b98c7b1f001,plaster_internal,A sample of plaster internal.,0.700,W/(m.K),950.00,kg/m3,,,False,,
1,mat_lib_id_01,material_id_102,solid_material,b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002,bricks_full,A sample of bricks full.,0.720,W/(m.K),1800.00,kg/m3,,,False,,
2,mat_lib_id_01,material_id_103,solid_material,b0a6a2d1-1003-4ff6-a7f9-1b98c7b1f003,bricks_perforated,A sample of bricks perforated.,0.450,W/(m.K),1400.00,kg/m3,,,False,,
3,mat_lib_id_01,material_id_104,solid_material,b0a6a2d1-1004-4ff6-a7f9-1b98c7b1f004,concrete_reinforced,A sample of concrete reinforced.,2.100,W/(m.K),2400.00,kg/m3,,,False,,
4,mat_lib_id_01,material_id_105,solid_material,b0a6a2d1-1005-4ff6-a7f9-1b98c7b1f005,concrete_lightweight,A sample of concrete lightweight.,0.600,W/(m.K),1100.00,kg/m3,,,False,,
5,mat_lib_id_01,material_id_106,solid_material,b0a6a2d1-1006-4ff6-a7f9-1b98c7b1f006,wood,A sample of wood.,0.130,W/(m.K),550.00,kg/m3,,,False,,
6,mat_lib_id_01,material_id_107,solid_material,b0a6a2d1-1007-4ff6-a7f9-1b98c7b1f007,insulation_eps,A sample of insulation eps.,0.035,W/(m.K),25.00,kg/m3,,,False,,
7,mat_lib_id_01,material_id_108,solid_material,b0a6a2d1-1008-4ff6-a7f9-1b98c7b1f008,insulation_rockwool,A sample of insulation rockwool.,0.036,W/(m.K),30.00,kg/m3,,,False,,
8,mat_lib_id_01,material_id_109,solid_material,b0a6a2d1-1009-4ff6-a7f9-1b98c7b1f009,glass,A sample of glass.,1.000,W/(m.K),2500.00,kg/m3,,,False,,
9,mat_lib_id_01,material_id_113,solid_material,b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010,plaster_external,A sample of plaster external.,0.930,W/(m.K),1050.00,kg/m3,,,False,,


In [15]:
# Checks
materials_path = r"C:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\IDL_files\materials_constructions\Materials.csv"
layers_path = r"C:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\IDL_files\materials_constructions\Layers.csv"
constructions_path = r"C:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\IDL_files\materials_constructions\LayeredConstructions.csv"

df_materials = pd.read_csv(materials_path)
df_layers = pd.read_csv(layers_path)
df_constructions = pd.read_csv(constructions_path)

df_materials = pd.read_csv(materials_path)
df_layers = pd.read_csv(layers_path)
df_constructions = pd.read_csv(constructions_path)

# Normalize and compare material IDs
df_materials['mat_lib_id_clean'] = df_materials['mat_lib_id'].astype(str).str.strip().str.lower()
df_layers['material_id_clean'] = df_layers['material_id'].astype(str).str.strip().str.lower()

material_ids_set = set(df_materials['mat_lib_id_clean'])
invalid_links = df_layers[~df_layers['material_id_clean'].isin(material_ids_set)]

# Count valid vs invalid
valid_count = len(df_layers) - len(invalid_links)
invalid_count = len(invalid_links)

valid_count, invalid_count



(414, 0)

CREATE GML FOR SENSORS

In [16]:
import os
import csv
import pandas as pd

folder_real_time_data = os.path.join(os.getcwd(), 'outputs', 'edts', 'simulation_data')

rt_file = []
for f in os.listdir(folder_real_time_data):
    if f.endswith('.csv') and os.path.isfile(os.path.join(folder_real_time_data, f)):
        rt_file.append(os.path.join(folder_real_time_data, f))

file_path = rt_file[0]

with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

start_idx = next(i for i, line in enumerate(lines) if line.startswith('#Sequence'))

df = pd.read_csv(
    file_path,
    sep=';',             
    skiprows=start_idx,   
    header=0,             
    decimal=',',        
    encoding='utf-8'
)

df.drop(columns=['Source', 'Type', 'Error/Time Change/Log Status', '#Sequence'], inplace=True, errors='ignore')
df.head(50)


,Date/Time,AS39001_AMB_A1_Tamb [°C],AS39001_AMB_A2_Tamb [°C],AS39001_AMB_A3_TAmb [°C],AS39001_AMB_A4_Tamb [°C],AS39001_AMB_A5_Tamb [°C],AS39001_AMB_A6_Tamb [°C],AS39001_AMB_A7_Tamb [°C]
0,13/10/2023 14:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13/10/2023 15:00,24.65,24.85,24.55,25.20,24.9,24.0,22.9
2,13/10/2023 15:30,24.65,25.10,24.80,25.45,25.3,24.0,22.7
3,13/10/2023 16:00,24.65,25.20,24.90,25.55,25.6,24.0,22.9
4,13/10/2023 16:30,24.65,25.20,24.90,25.75,25.7,24.0,22.9
5,13/10/2023 17:00,24.65,24.95,24.80,25.50,25.6,24.0,22.9
6,13/10/2023 17:30,24.65,24.80,24.80,25.10,25.3,24.0,22.9
7,13/10/2023 18:00,24.65,24.70,24.70,25.10,25.1,24.0,22.9
8,13/10/2023 18:30,24.65,24.55,24.70,24.95,24.9,24.0,22.9
9,13/10/2023 19:00,24.65,24.55,24.65,24.95,24.9,23.8,22.7


In [17]:
import uuid
import pandas as pd
from IPython.display import display, HTML

sensor_refs = [
    "AS39001_AMB_A1_Tamb [°C]",
    "AS39001_AMB_A2_Tamb [°C]",
    "AS39001_AMB_A3_TAmb [°C]",
    "AS39001_AMB_A4_Tamb [°C]",
    "AS39001_AMB_A5_Tamb [°C]",
    "AS39001_AMB_A6_Tamb [°C]",
    "AS39001_AMB_A7_Tamb [°C]"
]

# Grid layout
temp_x = [-37.12, -42.50, -50.34]
temp_y = [22.57, 13.8]
temp_positions = [(x, y) for x in temp_x for y in temp_y]  # 6 positions
z = 4.75

sensor_mapping = []

# First 6 sensors assigned to grid positions
for i in range(6):
    x, y = temp_positions[i]
    name = sensor_refs[i]
    sensor_mapping.append({
        "sensor": name,
        "sensor_id_UUID": f"{name}_{uuid.uuid5(uuid.NAMESPACE_DNS, f'sensor_{name}')}",
        "type": "temperature",
        "temperature": '[°C]',  # Same as sensor column — replace if mapping values
        "position": f"{x}, {y}, {z}"
    })

# Last sensor with custom position
name = sensor_refs[6]
custom_x, custom_y = -37.12, 13.86
sensor_mapping.append({
    "sensor": name,
    "sensor_id_UUID": f"{name}_{uuid.uuid5(uuid.NAMESPACE_DNS, f'sensor_{name}')}",
    "type": "temperature",
    "temperature": '[°C]',
    "position": f"{custom_x}, {custom_y}, {z}"
})

# Create DataFrame and reorder columns
df_mapping = pd.DataFrame(sensor_mapping)
df_mapping = df_mapping[["sensor", "sensor_id_UUID", "type", "temperature", "position"]]

# Display with wider columns
html = df_mapping.to_html(index=False)
styled_html = f"""
<style>
    table {{
        width: 100%;
        table-layout: fixed;
    }}
    th, td {{
        padding: 8px;
        text-align: center;
        word-wrap: break-word;
        font-size: 12px;
    }}
</style>
{html}
"""

display(HTML(styled_html))

mapping_output_path = os.path.join(folder_real_time_data, 'sensor_mapping.csv')
df_mapping.to_csv(mapping_output_path, index=False, encoding='utf-8')
print(f"✅ Exported sensor mapping to: {mapping_output_path}")


sensor,sensor_id_UUID,type,temperature,position
AS39001_AMB_A1_Tamb [°C],AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,temperature,[°C],"-37.12, 22.57, 4.75"
AS39001_AMB_A2_Tamb [°C],AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,temperature,[°C],"-37.12, 13.8, 4.75"
AS39001_AMB_A3_TAmb [°C],AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,temperature,[°C],"-42.5, 22.57, 4.75"
AS39001_AMB_A4_Tamb [°C],AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,temperature,[°C],"-42.5, 13.8, 4.75"
AS39001_AMB_A5_Tamb [°C],AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,temperature,[°C],"-50.34, 22.57, 4.75"
AS39001_AMB_A6_Tamb [°C],AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,temperature,[°C],"-50.34, 13.8, 4.75"
AS39001_AMB_A7_Tamb [°C],AS39001_AMB_A7_Tamb [°C]_dfe25541-50a9-53c1-b56f-0f77233b416a,temperature,[°C],"-37.12, 13.86, 4.75"


✅ Exported sensor mapping to: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\outputs\edts\simulation_data\sensor_mapping.csv


In [18]:
sensor_columns = [col for col in df.columns if col in df_mapping['sensor'].values]

uuid_row = [''] + [df_mapping.set_index('sensor').loc[col, 'sensor_id_UUID'] for col in sensor_columns]

df_with_uuid_header = pd.concat([
    pd.DataFrame([uuid_row], columns=['Date/Time'] + sensor_columns),
    df
], ignore_index=True)

# Save if needed
# df_with_uuid_header.to_csv("sensor_data_with_uuid_header.csv", index=False)

df_with_uuid_header.head(5)
# Make a copy of the original dataframe
df_with_uuids = df.copy()

# For each sensor, add a new column with the UUID
for _, row in df_mapping.iterrows():
    sensor_name = row['sensor']
    uuid = row['sensor_id_UUID']
    
    if sensor_name in df_with_uuids.columns:
        df_with_uuids[f'{sensor_name} UUID'] = uuid  # Same UUID repeated in the column

# Optional: Also add positions
for _, row in df_mapping.iterrows():
    sensor_name = row['sensor']
    pos = row['position']
    
    if sensor_name in df_with_uuids.columns:
        df_with_uuids[f'{sensor_name} Position'] = pos  # Position as string

# Step 1: Start with fixed columns (e.g. Date/Time)
interleaved_cols = [col for col in df.columns if col not in sensor_refs]

# Step 2: Interleave temperature and UUID columns
for sensor in sensor_refs:
    interleaved_cols.append(sensor)
    interleaved_cols.append(f"{sensor} UUID")

# Step 3: Create UUID columns in original DataFrame
for sensor in sensor_refs:
    uuid_value = df_mapping.loc[df_mapping['sensor'] == sensor, 'sensor_id_UUID'].values[0]
    df[f"{sensor} UUID"] = uuid_value

# Step 4: Reorder columns
df_interleaved = df[interleaved_cols]

df_interleaved.head(40)

sensor_refs = df_mapping['sensor'].tolist()

# Convert 'Date/Time' to datetime
df['Date/Time'] = pd.to_datetime(df['Date/Time'], dayfirst=True, errors='coerce')

# Split into 'Date' and 'Time'
df['Date'] = df['Date/Time'].dt.date
df['Time'] = df['Date/Time'].dt.time

# Add UUID columns to df
for sensor in sensor_refs:
    uuid_value = df_mapping.loc[df_mapping['sensor'] == sensor, 'sensor_id_UUID'].values[0]
    df[f"{sensor} UUID"] = uuid_value

# Reorder columns: Date, Time, then interleaved temperature and UUIDs
interleaved_cols = ['Date', 'Time']
for sensor in sensor_refs:
    interleaved_cols.extend([sensor, f"{sensor} UUID"])

# Create interleaved DataFrame
df_interleaved = df[interleaved_cols]

df_interleaved.head(40)


,Date,Time,AS39001_AMB_A1_Tamb [°C],AS39001_AMB_A1_Tamb [°C] UUID,AS39001_AMB_A2_Tamb [°C],AS39001_AMB_A2_Tamb [°C] UUID,AS39001_AMB_A3_TAmb [°C],AS39001_AMB_A3_TAmb [°C] UUID,AS39001_AMB_A4_Tamb [°C],AS39001_AMB_A4_Tamb [°C] UUID,AS39001_AMB_A5_Tamb [°C],AS39001_AMB_A5_Tamb [°C] UUID,AS39001_AMB_A6_Tamb [°C],AS39001_AMB_A6_Tamb [°C] UUID,AS39001_AMB_A7_Tamb [°C],AS39001_AMB_A7_Tamb [°C] UUID
0,2023-10-13,14:35:00,NaN,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,NaN,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,NaN,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,NaN,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,NaN,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,NaN,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,NaN,AS39001_AMB_A7_Tamb [°C]_dfe25541-50a9-53c1-b56f-0f77233b416a
1,2023-10-13,15:00:00,24.65,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,24.85,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,24.55,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,25.20,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,24.9,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,24.0,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,22.9,AS39001_AMB_A7_Tamb [°C]_dfe25541-50a9-53c1-b56f-0f77233b416a
2,2023-10-13,15:30:00,24.65,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,25.10,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,24.80,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,25.45,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,25.3,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,24.0,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,22.7,AS39001_AMB_A7_Tamb [°C]_dfe25541-50a9-53c1-b56f-0f77233b416a
3,2023-10-13,16:00:00,24.65,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,25.20,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,24.90,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,25.55,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,25.6,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,24.0,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,22.9,AS39001_AMB_A7_Tamb [°C]_dfe25541-50a9-53c1-b56f-0f77233b416a
4,2023-10-13,16:30:00,24.65,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,25.20,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,24.90,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,25.75,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,25.7,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,24.0,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,22.9,AS39001_AMB_A7_Tamb [°C]_dfe25541-50a9-53c1-b56f-0f77233b416a
5,2023-10-13,17:00:00,24.65,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,24.95,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,24.80,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,25.50,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,25.6,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,24.0,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,22.9,AS39001_AMB_A7_Tamb [°C]_dfe25541-50a9-53c1-b56f-0f77233b416a
6,2023-10-13,17:30:00,24.65,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,24.80,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,24.80,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,25.10,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,25.3,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,24.0,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,22.9,AS39001_AMB_A7_Tamb [°C]_dfe25541-50a9-53c1-b56f-0f77233b416a
7,2023-10-13,18:00:00,24.65,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,24.70,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d

In [19]:


# # 2. Interleave sensor and UUID columns
# sensor_columns = [col for col in df.columns if col in df_mapping['sensor'].values]
# uuid_map = df_mapping.set_index('sensor')['sensor_id_UUID'].to_dict()

# # Add UUID columns to df
# for sensor in sensor_columns:
#     df[f"{sensor} UUID"] = uuid_map[sensor]

# # Build final interleaved column order
# interleaved_cols = ['Date', 'Time']
# for sensor in sensor_columns:
#     interleaved_cols.append(sensor)
#     interleaved_cols.append(f"{sensor} UUID")

# df = df[interleaved_cols]

# # 3. Build UUID row that matches final columns
# uuid_row = []
# for col in df.columns:
#     if col.endswith("UUID"):
#         uuid_row.append(df[col].iloc[0])  # Grab actual UUID value
#     elif col in sensor_columns:
#         uuid_row.append('')  # Temperature column — no UUID here
#     else:
#         uuid_row.append('')  # Date/Time columns — no UUID

# # 4. Insert UUID row
# df_final = pd.concat([
#     pd.DataFrame([uuid_row], columns=df.columns),
#     df
# ], ignore_index=True)

# df_final = df_final.drop(columns = ['AS39001_AMB_A7_Tamb [°C]', 'AS39001_AMB_A7_Tamb [°C] UUID']


# 1. Prepare sensor column list and UUID mapping
sensor_columns = [col for col in df.columns if col in df_mapping['sensor'].values]
uuid_map = df_mapping.set_index('sensor')['sensor_id_UUID'].to_dict()

# 2. Add UUID columns to df
for sensor in sensor_columns:
    df[f"{sensor} UUID"] = uuid_map[sensor]

# 3. Build final interleaved column order — UUID first, then temperature
interleaved_cols = ['Date', 'Time']
for sensor in sensor_columns:
    interleaved_cols.append(f"{sensor} UUID")  # UUID first
    interleaved_cols.append(sensor)            # Then temperature

# 4. Reorder DataFrame
df = df[interleaved_cols]

# 5. Build UUID row (UUIDs only in UUID columns)
uuid_row = []
for col in df.columns:
    if col.endswith("UUID"):
        uuid_row.append(df[col].iloc[0])
    else:
        uuid_row.append('')

# 6. Insert UUID row
df_final = pd.concat([
    pd.DataFrame([uuid_row], columns=df.columns),
    df
], ignore_index=True)

# 7. Drop unwanted columns
df_final.drop(columns=[
    'AS39001_AMB_A7_Tamb [°C] UUID', 'AS39001_AMB_A7_Tamb [°C]'
], inplace=True, errors='ignore')

# 8. Add line counter as the first column (starting from 0 including UUID row)
df_final.insert(0, "Line", range(len(df_final)))

# 9. Display result
from IPython.display import display, HTML
display(HTML(df_final.head(20).to_html(index=False)))




Line,Date,Time,AS39001_AMB_A1_Tamb [°C] UUID,AS39001_AMB_A1_Tamb [°C],AS39001_AMB_A2_Tamb [°C] UUID,AS39001_AMB_A2_Tamb [°C],AS39001_AMB_A3_TAmb [°C] UUID,AS39001_AMB_A3_TAmb [°C],AS39001_AMB_A4_Tamb [°C] UUID,AS39001_AMB_A4_Tamb [°C],AS39001_AMB_A5_Tamb [°C] UUID,AS39001_AMB_A5_Tamb [°C],AS39001_AMB_A6_Tamb [°C] UUID,AS39001_AMB_A6_Tamb [°C]
0,,,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,
1,2023-10-13,14:35:00,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,NaN,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,NaN,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,NaN,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,NaN,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,NaN,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,NaN
2,2023-10-13,15:00:00,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,24.65,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,24.85,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,24.55,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,25.2,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,24.9,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,24.0
3,2023-10-13,15:30:00,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,24.65,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,25.1,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,24.8,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,25.45,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,25.3,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,24.0
4,2023-10-13,16:00:00,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,24.65,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,25.2,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,24.9,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,25.55,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,25.6,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,24.0
5,2023-10-13,16:30:00,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,24.65,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,25.2,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,24.9,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,25.75,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,25.7,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,24.0
6,2023-10-13,17:00:00,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,24.65,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,24.95,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,24.8,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,25.5,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,25.6,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,24.0
7,2023-10-13,17:30:00,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,24.65,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,24.8,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,24.8,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,25.1,AS39001_AMB_A5_Tamb [°C]_9050abaa-afea-53d8-89de-72d8d3bd8223,25.3,AS39001_AMB_A6_Tamb [°C]_2b225635-50cd-5513-8f29-57038e594fba,24.0
8,2023-10-13,18:00:00,AS39001_AMB_A1_Tamb [°C]_ae348efb-ce80-5f03-9903-da31a2fbc8de,24.65,AS39001_AMB_A2_Tamb [°C]_79b705b5-6ac0-5dc7-99d0-ab78d21d8d25,24.7,AS39001_AMB_A3_TAmb [°C]_aee72039-ea89-5502-ad74-9570e14be978,24.7,AS39001_AMB_A4_Tamb [°C]_d6cccc5f-fd24-5a84-82fe-af7bbf4e76d2,25.1,AS39001_

In [20]:
output_path = os.path.join(folder_real_time_data, 'temperature_sensors_uuid.csv')

df_final.to_csv(output_path, index=False, encoding='utf-8')
print(f"✅ Exported successfully to:\n{output_path}")


✅ Exported successfully to:
c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\outputs\edts\simulation_data\temperature_sensors_uuid.csv


In [21]:
overlapping_visual_check = True

if overlapping_visual_check:
    import plotly.graph_objects as go
    import numpy as np

    fig = go.Figure()

    # def match_sensor_name(x, y, df_mapping, tolerance=0.5):
    #     x, y = round(x, 2), round(y, 2)
    #     for _, row in df_mapping.iterrows():
    #         pos_parts = row['position'].split(", ")
    #         if len(pos_parts) >= 2:
    #             px, py = round(float(pos_parts[0]), 2), round(float(pos_parts[1]), 2)
    #             if abs(px - x) <= tolerance and abs(py - y) <= tolerance:
    #                 return row['sensor'].replace(" [°C]", "")
    #     return None

    def match_sensor_name(x, y, df_mapping, tolerance=0.5):
        x, y = round(x, 2), round(y, 2)
        for _, row in df_mapping.iterrows():
            pos_parts = row['position'].split(", ")
            if len(pos_parts) >= 2:
                px, py = round(float(pos_parts[0]), 2), round(float(pos_parts[1]), 2)
                if abs(px - x) <= tolerance and abs(py - y) <= tolerance:
                    # Clean name: remove " [°C]" and any trailing "_Tamb" or "_TAmb"
                    name = row['sensor'].replace(" [°C]", "")
                    if name.lower().endswith("_tamb"):
                        name = name[:-5]
                    return name
        return None


    def draw_bbox(fig, min_xyz, max_xyz, color="red", name="BBox"):
        x0, y0, z0 = min_xyz
        x1, y1, z1 = max_xyz

        corners = np.array([
            [x0, y0, z0],
            [x1, y0, z0],
            [x1, y1, z0],
            [x0, y1, z0],
            [x0, y0, z1],
            [x1, y0, z1],
            [x1, y1, z1],
            [x0, y1, z1]
        ])

        edges = [
            (0, 1), (1, 2), (2, 3), (3, 0),
            (4, 5), (5, 6), (6, 7), (7, 4),
            (0, 4), (1, 5), (2, 6), (3, 7)
        ]

        for i, j in edges:
            fig.add_trace(go.Scatter3d(
                x=[corners[i][0], corners[j][0]],
                y=[corners[i][1], corners[j][1]],
                z=[corners[i][2], corners[j][2]],
                mode="lines",
                line=dict(color=color, width=2),
                showlegend=False,
                name=name
            ))

    # === Plot thematic walls ===
    for _, row in df_thematic_walls.iterrows():
        coords = np.array(row["surface_vertices"])
        fig.add_trace(go.Scatter3d(
            x=coords[:, 0], y=coords[:, 1], z=coords[:, 2],
            mode="lines",
            line=dict(color="blue", width=3),
            name=f"Thematic: {row['wall_type']}",
            hoverinfo="text",
            text=f"ID: {row['element_id']}<br>Type: {row['wall_type']}<br>Th: {row.get('thickness_rounded', '')}"
        ))

    # === Plot bounding boxes ===
    for wall in walls_info:
        min_xyz, max_xyz = get_bbox(wall["surface_vertices"])
        draw_bbox(fig, min_xyz, max_xyz,
                  color="red",
                  name=f"BBox Wall {wall['wall_index']}")

    # === Sensor positions (rounded) ===
    temp_x = [-37.00, -42.50, -50.00]
    temp_y = [22.50, 13.80]

    temp_positions = [(round(x, 2), round(y, 2)) for x in temp_x for y in temp_y]

    sensors = (
        [{"id": f"temp_{x}_{y}", "type": "temperature", "position": [x, y, 4.70]} for x, y in temp_positions])

    # --- Build position-to-name mapping using df_mapping ---
    position_to_clean_name = {
        tuple(round(float(c), 2) for c in row['position'].split(", ")[:2]):  # round x, y
        row['sensor'].replace(" [°C]", "")  # clean name
        for _, row in df_mapping.iterrows()
    }

    # --- Plot sensors with clean names ---
    for sensor in sensors:
        x, y, z = sensor["position"]
        label = match_sensor_name(x, y, df_mapping)
        label = label if label else sensor["id"]  # fallback if no match

        fig.add_trace(go.Scatter3d(
            x=[x], y=[y], z=[z],
            mode="markers+text",
            marker=dict(size=5, color="orange" if sensor["type"] == "temperature" else "green"),
            text=[label],
            textfont=dict(
                size= 8,       # adjust smaller text size
                family="Verdana",  # or try "Arial", "Courier New", etc.
                color="black"
            ),
            name=sensor["type"].capitalize(),
            showlegend=False
        ))


    # === Layout ===
    fig.update_layout(
        title="Overlap of Thematic Surfaces, Wall Bounding Boxes, and Sensors",
        scene=dict(
            xaxis_title="x", yaxis_title="y", zaxis_title="z",
            aspectmode="data",
            bgcolor="white"
        ),
        legend=dict(itemsizing="constant"),
        margin=dict(l=10, r=10, t=40, b=10),
        width=1400, height=800
    )

    fig.show()


In [22]:
library_df

,library_id,class,gmlid,name,description,librarycode,librarycode_codespace,glazing_ratio,glazing_ratio_uom,uvalue,uvalue_uom,transparency_fraction,emissivity_fraction,emissivity_side,reflectance_fraction,reflectance_wawelength_range,transmittance_fraction,transmittance_side,transmittance_wawelength_range,layered_construction_id
0,LayConst_lib_id_01,LayeredConstruction,Layer_id_a48d94d6-2f7d-4299-998e-d91b72d00fd9,external_wall_brick_10cm,external_wall with total thickness 125 mm,,,,,5.999,W/(K.m2),,,,,,,,,LayeredConstr_id_0001
1,LayConst_lib_id_01,LayeredConstruction,Layer_id_167d271a-ded9-4b2c-8742-3c795cf062bc,external_wall_brick_12cm,external_wall with total thickness 150 mm,,,,,4.965,W/(K.m2),,,,,,,,,LayeredConstr_id_0002
2,LayConst_lib_id_01,LayeredConstruction,Layer_id_c07d0f0d-004d-4260-b55f-506b37b9ee89,external_wall_brick_15cm,external_wall with total thickness 175 mm,,,,,4.235,W/(K.m2),,,,,,,,,LayeredConstr_id_0003
3,LayConst_lib_id_01,LayeredConstruction,Layer_id_43d4fd91-b8e6-49ca-a947-4b4ca5f757e6,external_wall_brick_17cm,external_wall with total thickness 200 mm,,,,,3.692,W/(K.m2),,,,,,,,,LayeredConstr_id_0004
4,LayConst_lib_id_01,LayeredConstruction,Layer_id_9da8efb6-4050-4b7a-870e-3c3a0f4722de,external_wall_brick_20cm,external_wall with total thickness 225 mm,,,,,3.272,W/(K.m2),,,,,,,,,LayeredConstr_id_0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,LayConst_lib_id_01,LayeredConstruction,Layer_id_13520522-7c1d-47a9-a610-c9f516f6716a,door_wooden_4cm,door with total thickness 40 mm,,,,,3.967,W/(K.m2),,,,,,,,,LayeredConstr_id_0107
107,LayConst_lib_id_01,LayeredConstruction,Layer_id_77d6bed2-a210-4517-bbfb-2909282994aa,ceiling_concrete_plastered,ceiling with total thickness 250 mm,,,,,6.777,W/(K.m2),,,,,,,,,LayeredConstr_id_0108
108,LayConst_lib_id_01,LayeredConstruction,Layer_id_8692653e-b8e4-4a42-bd2a-d38d667a9ae5,floor_concrete_wood_finish,floor with total thickness 300 mm,,,,,3.431,W/(K.m2),,,,,,,,,LayeredConstr_id_0109
109,LayConst_lib_id_01,LayeredConstruction,Layer_id_babfe70f-b162-486e-bb29-abd87dd28ca2,window_triple_glass,window with total thickness 45 mm,,,0.5,,1.200,W/(K.m2),0.65,,,,,,,,LayeredConstr_id_0110


In [ ]:

internal_walls = df_thematic_surfaces[df_thematic_surfaces['thematic_surface'] == 'internal_wall']
internal_thicknesses_round = internal_walls['thickness_rounded']

external_walls = df_thematic_surfaces[df_thematic_surfaces['thematic_surface'] == 'external_wall']
external_thicknesses_round = external_walls['thickness_rounded']

library_df

wall_lib_th = library_df[library_df'']



In [ ]:




for e in library_df.interrows():

    lay_con = library_df.copy()
    name = lay_con['name']
    ext_wall = [name]['external'].split('_')
    int_wall =[name]['internal'].split('_')
    walls = ext_wall + int_wall
    if ext_wall in walls:

        gml_id =
        th = 
        u = 
        
    if 


def assign_uuid_to_walls(walls):

    walls_th_uuid = {}

    





In [23]:
df_thematic_surfaces

,element_id,parent_id,thematic_surface,wall_type,has_holes,surface_vertices,centroid,matched_thickness,thickness_rounded
0,2,1.0,internal_wall,internal_wall,False,"[(-39.014, 22.381, 6.657), (-39.119, 29.884, 6.657), (-39.119, 29.884, 3.052), (-39.014, 22.381, 3.052), (-39.014, 22.381, 6.657)]","[-39.056000000000004, 25.3822, 5.215]",0.259339,0.250
1,3,1.0,external_wall,external_wall,False,"[(-39.119, 29.884, 6.657), (-31.666, 29.983, 6.657), (-31.666, 29.983, 3.052), (-39.119, 29.884, 3.052), (-39.119, 29.884, 6.657)]","[-36.1378, 29.9236, 5.215]",0.644637,0.650
2,5,1.0,external_wall,external_wall,False,"[(-31.666, 29.983, 3.052), (-31.666, 29.983, 6.657), (-31.513, 22.966, 6.657), (-31.513, 22.966, 3.052), (-31.666, 29.983, 3.052)]","[-31.6048, 27.1762, 4.494]",0.343848,0.350
3,6,1.0,external_wall,external_wall,False,"[(-31.513, 22.966, 3.052), (-31.513, 22.966, 6.657), (-31.502, 22.488, 6.657), (-31.502, 22.488, 3.052), (-31.513, 22.966, 3.052)]","[-31.5086, 22.7748, 4.494]",0.150000,NaN
4,7,1.0,internal_wall,internal_wall,False,"[(-31.502, 22.488, 6.657), (-39.014, 22.381, 6.657), (-39.014, 22.381, 3.052), (-31.502, 22.488, 3.052), (-31.502, 22.488, 6.657)]","[-34.506800000000005, 22.4452, 5.215]",0.281342,0.275
...,...,...,...,...,...,...,...,...,...
170,217,215.0,internal_wall,internal_wall,False,"[(-51.577, 18.699, 3.052), (-51.577, 18.699, 6.657), (-51.608, 19.866, 6.657), (-51.608, 19.866, 3.052), (-51.577, 18.699, 3.052)]","[-51.5894, 19.165799999999997, 4.494]",0.161408,0.150
171,218,215.0,internal_wall,internal_wall,False,"[(-51.608, 19.866, 3.052), (-51.608, 19.866, 6.657), (-49.906, 19.893, 6.657), (-49.906, 19.893, 3.052), (-51.608, 19.866, 3.052)]","[-50.9272, 19.8768, 4.494]",0.194037,0.200
172,219,215.0,internal_wall,internal_wall,False,"[(-49.906, 19.893, 3.052), (-49.906, 19.893, 6.657), (-49.892, 18.724, 6.657), (-49.892, 18.724, 3.052), (-49.906, 19.893, 3.052)]","[-49.900400000000005, 19.425400000000003, 4.494]",0.126659,0.125
173,221,215.0,ceiling,ceiling,False,"[(-49.892, 18.724, 6.657), (-49.906, 19.893, 6.657), (-51.608, 19.866, 6.657), (-51.577, 18.699, 6.657), (-49.892, 18.724, 6.657)]","[-50.575, 19.1812, 6.656999999999999]",0.300000,0.300


In [ ]:
fffffff

NameError: name 'fffffff' is not defined

In [ ]:
# LAYERED CONSTRUCTIONS

pd.set_option('display.max_colwidth', 600)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)

layers_df.head(80)

from IPython.display import display, HTML
display(HTML(layers_df.head(80).to_html()))

,library_id,parent_id,gmlid,name,description,thickness,thickness_uom,material_id,material_reference
0,LayConstLib_id_02,LayeredConstr_id_0001,Layer_id_0a9496c6-1189-4b97-92e8-050babbf7e19,plaster_external,"plaster_external layer, 12 mm thick",12.5,mm,mat_lib_id_113,material_id_b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010
1,LayConstLib_id_02,LayeredConstr_id_0001,Layer_id_45d1524b-16ed-4d1e-9fba-34548f9fe247,bricks_full,"bricks_full layer, 100 mm thick",100.0,mm,mat_lib_id_102,material_id_b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002
2,LayConstLib_id_02,LayeredConstr_id_0001,Layer_id_69388852-0b98-47dd-8fc1-5e4460c0c140,mortar,"mortar layer, 12 mm thick",12.5,mm,material_id_material_id_202,material_id_b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012
3,LayConstLib_id_02,LayeredConstr_id_0002,Layer_id_0f382e36-6838-4ebd-8325-0a68498a0fe1,plaster_external,"plaster_external layer, 12 mm thick",12.5,mm,mat_lib_id_113,material_id_b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010
4,LayConstLib_id_02,LayeredConstr_id_0002,Layer_id_b930778d-b31d-460b-83d8-6f86c7716aa8,bricks_full,"bricks_full layer, 125 mm thick",125.0,mm,mat_lib_id_102,material_id_b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002
5,LayConstLib_id_02,LayeredConstr_id_0002,Layer_id_609ccb48-a96b-4c37-874f-d9573938be90,mortar,"mortar layer, 12 mm thick",12.5,mm,material_id_material_id_202,material_id_b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012
6,LayConstLib_id_02,LayeredConstr_id_0003,Layer_id_f1611e8b-b8cc-4a8a-8c8a-0fe20eef00bd,plaster_external,"plaster_external layer, 12 mm thick",12.5,mm,mat_lib_id_113,material_id_b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010
7,LayConstLib_id_02,LayeredConstr_id_0003,Layer_id_4aafdb53-d5c2-4aff-8ee1-8f162e9bb50c,bricks_full,"bricks_full layer, 150 mm thick",150.0,mm,mat_lib_id_102,material_id_b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002
8,LayConstLib_id_02,LayeredConstr_id_0003,Layer_id_98333f68-1876-46ae-b7cf-0944c21aa427,mortar,"mortar layer, 12 mm thick",12.5,mm,material_id_material_id_202,material_id_b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012
9,LayConstLib_id_02,LayeredConstr_id_0004,Layer_id_84b5e0d0-ca77-490e-826d-bbd02259b970,plaster_external,"plaster_external layer, 12 mm thick",12.5,mm,mat_lib_id_113,material_id_b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010


In [ ]:
# MATERIAL LIBRARY 

pd.set_option('display.max_colwidth', 600)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
# formatted_df.head(60)
formatted_df.head(40)


,library_id,mat_lib_id,class,gml_id,name,description,thermal_conductivity,thermal_conductivity_uom,density,density_uom,spec_heat_cap,spec_heat_cap_uom,is_ventilated,r_value,r_value_uom
0,mat_lib_id_01,material_id_101,solid_material,b0a6a2d1-1001-4ff6-a7f9-1b98c7b1f001,plaster_internal,A sample of plaster internal.,0.700,W/(m.K),950.00,kg/m3,,,False,,
1,mat_lib_id_01,material_id_102,solid_material,b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002,bricks_full,A sample of bricks full.,0.720,W/(m.K),1800.00,kg/m3,,,False,,
2,mat_lib_id_01,material_id_103,solid_material,b0a6a2d1-1003-4ff6-a7f9-1b98c7b1f003,bricks_perforated,A sample of bricks perforated.,0.450,W/(m.K),1400.00,kg/m3,,,False,,
3,mat_lib_id_01,material_id_104,solid_material,b0a6a2d1-1004-4ff6-a7f9-1b98c7b1f004,concrete_reinforced,A sample of concrete reinforced.,2.100,W/(m.K),2400.00,kg/m3,,,False,,
4,mat_lib_id_01,material_id_105,solid_material,b0a6a2d1-1005-4ff6-a7f9-1b98c7b1f005,concrete_lightweight,A sample of concrete lightweight.,0.600,W/(m.K),1100.00,kg/m3,,,False,,
5,mat_lib_id_01,material_id_106,solid_material,b0a6a2d1-1006-4ff6-a7f9-1b98c7b1f006,wood,A sample of wood.,0.130,W/(m.K),550.00,kg/m3,,,False,,
6,mat_lib_id_01,material_id_107,solid_material,b0a6a2d1-1007-4ff6-a7f9-1b98c7b1f007,insulation_eps,A sample of insulation eps.,0.035,W/(m.K),25.00,kg/m3,,,False,,
7,mat_lib_id_01,material_id_108,solid_material,b0a6a2d1-1008-4ff6-a7f9-1b98c7b1f008,insulation_rockwool,A sample of insulation rockwool.,0.036,W/(m.K),30.00,kg/m3,,,False,,
8,mat_lib_id_01,material_id_109,solid_material,b0a6a2d1-1009-4ff6-a7f9-1b98c7b1f009,glass,A sample of glass.,1.000,W/(m.K),2500.00,kg/m3,,,False,,
9,mat_lib_id_01,material_id_113,solid_material,b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010,plaster_external,A sample of plaster external.,0.930,W/(m.K),1050.00,kg/m3,,,False,,


In [ ]:
cccccc

NameError: name 'cccccc' is not defined

In [ ]:
window_keywords = ["window", "glass", "glazed"]
pattern = "|".join(window_keywords)
print(library_df[library_df['name'].str.lower().str.contains(pattern)][['name', 'glazing_ratio', 'uvalue', 'uvalue_uom']])


                    name glazing_ratio  uvalue uvalue_uom
105  window_double_glass          0.65     2.8   W/(K.m2)
109  window_triple_glass           0.5     1.2   W/(K.m2)
110  window_single_glass           0.6     3.6   W/(K.m2)


In [ ]:
xxxx

NameError: name 'xxxx' is not defined

In [ ]:
from ast import literal_eval
import numpy as np

material_lambda_lookup = {
    mat["name"]: mat.get("lambda_W_per_mK", np.nan)
    for mat in materials_library
}

material_uuid_lookup = {
    row["name"]: row["uuid"] for _, row in df_materials.iterrows()
}

def assign_material_uuids(layers):
    try:
        if isinstance(layers, str):
            layers = literal_eval(layers)
        if not isinstance(layers, list):
            return []
        for layer in layers:
            material_name = layer.get("material")
            layer["material_uuid"] = material_uuid_lookup.get(material_name)
        return layers
    except Exception:
        return []

def compute_u_and_lambda_mean(layers):
    try:
        if isinstance(layers, str):
            layers = literal_eval(layers)
        if not isinstance(layers, list):
            return np.nan, np.nan

        r_total = 0.0
        lambdas = []
        for layer in layers:
            material = layer.get("material")
            thickness = layer.get("thickness")
            lam = material_lambda_lookup.get(material)
            if material and thickness and lam and lam > 0:
                r_total += thickness / lam
                lambdas.append(lam)

        u_value = round(1.0 / r_total, 3) if r_total > 0 else np.nan
        lambda_mean = round(np.mean(lambdas), 3) if lambdas else np.nan
        return u_value, lambda_mean
    except Exception:
        return np.nan, np.nan

df_constructions["layers"] = df_constructions["layers"].apply(assign_material_uuids)

df_constructions[["u_value", "lambda_mean"]] = df_constructions["layers"].apply(
    lambda x: pd.Series(compute_u_and_lambda_mean(x))
)



In [ ]:
pd.set_option("display.max_columns", None)        # show all columns
pd.set_option("display.max_colwidth", None)       # don't truncate column content
pd.set_option("display.width", 2000)              # expand total width for better formatting

df_constructions.head(60)

,name,wall_type,match_thickness,layers,construction_uuid,u_value,lambda_mean
0,external_wall_brick_10cm,external_wall,0.125,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.1}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",56e0a038-42b7-4291-8acf-7a0054e48950,5.999,0.840
1,external_wall_brick_12cm,external_wall,0.150,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.125}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",42bc659f-a7be-47da-90e9-7b66fa6295a2,4.965,0.840
2,external_wall_brick_15cm,external_wall,0.175,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.15}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",941f78eb-b5b5-4fcc-a714-e865cc42d0c6,4.235,0.840
3,external_wall_brick_17cm,external_wall,0.200,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.175}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",18060311-ac66-441f-a33e-40278270dc87,3.692,0.840
4,external_wall_brick_20cm,external_wall,0.225,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.2}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",abded65c-0191-4d75-ab74-84f69534ea0e,3.272,0.840
5,external_wall_brick_25cm,external_wall,0.275,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.25}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",b007276f-03ab-445a-9905-c5319fb06681,2.666,0.840
6,external_wall_brick_30cm,external_wall,0.325,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.3}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",f5d8ef6f-e8ae-45ff-a417-8e8a96c8e9a4,2.250,0.840
7,internal_wall_brick_10cm,internal_wall,0.125,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.1}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",000d14aa-308e-4ad2-bed0-719af975fe50,5.999,0.840
8,internal_wall_brick_12cm,internal_wall,0.150,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.125}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",4a626d94-771c-4007-8c09-bd853dd856ad,4.965,0.840
9,internal_wall_brick_15cm,internal_wall,0.175,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {

In [ ]:
import pandas as pd
import numpy as np
import uuid
from ast import literal_eval
from shapely.geometry import Polygon, MultiPolygon
from shapely.wkt import loads as load_wkt
from shapely.geometry.base import BaseGeometry

def safe_load_wkt(val):
    try:
        if isinstance(val, BaseGeometry):
            return val
        if isinstance(val, str) and val.strip():
            geom = load_wkt(val)
            return MultiPolygon([geom]) if isinstance(geom, Polygon) else geom
    except Exception:
        return None
    return None

if "geometry" not in df.columns:
    df["geometry"] = df["geometry_wkt"].apply(safe_load_wkt)

surface_types_to_extract = [
    "external", "external_with_hole", "internal", "internal_with_hole", "partly_wall", "ceiling", "floor", "window", "door"
]

df_surfaces = df[df["thematic_surface"].isin(surface_types_to_extract)]

assign_thickness_step_02 = True

if assign_thickness_step_02:
    # --- Bounding box & centroid utilities ---
    def get_bbox(surface_vertices):
        vertices = np.array(surface_vertices)
        min_xyz = vertices.min(axis=0)
        max_xyz = vertices.max(axis=0)
        return min_xyz, max_xyz

    def get_centroid(coords):
        coords = np.array(coords)
        return coords.mean(axis=0)

    def find_thickness(centroid, wall_bboxes):
        for bbox in wall_bboxes:
            if np.all(centroid >= bbox["min_xyz"]) and np.all(centroid <= bbox["max_xyz"]):
                return bbox["thickness"]
        return np.nan

    def assign_standard_thickness(row):
        wall_type = str(row.get("wall_type", "")).strip().lower()
        if pd.isna(row["matched_thickness"]):
            if wall_type == "external":
                return 0.35
            elif wall_type == "internal":
                return 0.15
            elif wall_type == "partly":
                return 0.125
        return row["matched_thickness"]

    round_thickness_value = 0.025

    # --- Prepare wall bounding boxes ---
    wall_bboxes = []
    for wall in walls_info:
        min_xyz, max_xyz = get_bbox(wall["surface_vertices"])
        wall_bboxes.append({
            "wall_index": wall["wall_index"],
            "thickness": wall["thickness"],
            "min_xyz": min_xyz,
            "max_xyz": max_xyz
        })

    # --- Assign centroids & match thickness ---
    df_thematic_walls["centroid"] = df_thematic_walls["surface_vertices"].apply(get_centroid)
    df_thematic_walls["matched_thickness"] = df_thematic_walls["centroid"].apply(
        lambda c: find_thickness(c, wall_bboxes)
    )
    df_thematic_walls["matched_thickness"] = df_thematic_walls.apply(assign_standard_thickness, axis=1)

    df_thematic_walls["thickness_rounded"] = df_thematic_walls["matched_thickness"].apply(
        lambda t: round(t / round_thickness_value) * round_thickness_value if pd.notna(t) else np.nan
    )

    # --- Flatten surface geometries ---
    thematic_surfaces = []
    for _, row in df_surfaces.iterrows():
        if not isinstance(row["geometry"], (Polygon, MultiPolygon)):
            continue
        geoms = [row["geometry"]] if isinstance(row["geometry"], Polygon) else row["geometry"].geoms
        for poly in geoms:
            coords = list(poly.exterior.coords)
            thematic_surfaces.append({
                "element_id": row["element_id"],
                "parent_id": row["parent_id"],
                "thematic_surface": row["thematic_surface"],
                "has_holes": row.get("has_holes", False),
                "surface_vertices": coords
            })

    df_thematic_surfaces = pd.DataFrame(thematic_surfaces)

    def normalize_surface_type(st):
        st = str(st).strip().lower()
        if st in {"external", "external_wall", "exterior", "outside", "external_with_hole"}:
            return "external_wall"
        elif st in {"internal", "internal_wall", "inside", "internal_with_hole"}:
            return "internal_wall"
        elif st in {"partly", "partlywall", "semi_internal", "semi-exposed"}:
            return "partly_wall"
        elif st in {"ceiling", "roof"}:
            return "ceiling"
        elif st == "floor":
            return "floor"
        elif "window" in st:
            return "window"
        elif "door" in st:
            return "door"
        return st

    # --- Default thickness by type ---
    def assign_default_thickness_general(row):
        if pd.notna(row["matched_thickness"]):
            return row["matched_thickness"]
        default_thickness_map = {
            "external_wall": 0.35,
            "internal_wall": 0.15,
            "partly_wall": 0.125,
            "ceiling": 0.25,
            "floor": 0.30,
            "window": 0.05,
            "door": 0.04
        }
        return default_thickness_map.get(row["surface_type"], np.nan)

    def assign_core_material(stype):
        if "wall" in stype:
            return "bricks"
        elif stype == "ceiling":
            return "concrete"
        elif stype == "floor":
            return "concrete"
        elif stype == "window":
            return "glass"
        elif stype == "door":
            return "wood"
        return "unknown"

    def assign_construction_name(row):
        stype = row.get("surface_type", "unknown")
        cm = row.get("core_material", "unknown")
        tr_val = row.get("thickness_rounded", np.nan)

        if pd.isna(tr_val):
            return f"{stype}_{cm}_unknown"
        
        tr = int(round(tr_val * 100))  # convert to cm
        return f"{stype}_{cm}_{tr}cm"

    def normalize_construction_name(name):
        return (
            str(name)
            .lower()
            .replace("bricks_", "brick_")
            .replace("bricks", "brick")
            .replace("concretes_", "concrete_")
            .replace("concretes", "concrete")
            .replace("  ", " ")
            .strip()
        )

    def extract_layer_summary(layers):
        try:
            if isinstance(layers, str):
                layers = literal_eval(layers)
            if not layers or not isinstance(layers, list):
                return None
            return " | ".join(f"{l['material']} ({l.get('thickness', '?')}m)" for l in layers if 'material' in l)
        except Exception:
            return None

    # def generate_default_layers(stype, thickness):
    #     if pd.isna(thickness):
    #         return []

    #     def get_uuid(material_name):
    #         return material_lookup.get(material_name.lower())

    #     if stype in ["external_wall", "internal_wall", "partly_wall"]:
    #         core_thickness = round(max(thickness - 0.025, 0.05), 5)
    #         return [
    #             {"material": "plaster_external", "material_uuid": get_uuid("plaster_external"), "thickness": 0.0125},
    #             {"material": "bricks_full", "material_uuid": get_uuid("bricks_full"), "thickness": core_thickness},
    #             {"material": "mortar", "material_uuid": get_uuid("mortar"), "thickness": 0.0125}
    #         ]
    #     elif stype == "ceiling":
    #         return [
    #             {"material": "plaster_internal", "material_uuid": get_uuid("plaster_internal"), "thickness": 0.015},
    #             {"material": "concrete", "material_uuid": get_uuid("concrete"), "thickness": round(thickness - 0.03, 5)},
    #             {"material": "plaster_external", "material_uuid": get_uuid("plaster_external"), "thickness": 0.015}
    #         ]
    #     elif stype == "floor":
    #         return [
    #             {"material": "wood_parquet", "material_uuid": get_uuid("wood_parquet"), "thickness": 0.02},
    #             {"material": "screed", "material_uuid": get_uuid("screed"), "thickness": 0.05},
    #             {"material": "concrete", "material_uuid": get_uuid("concrete"), "thickness": round(thickness - 0.07, 5)}
    #         ]
    #     elif stype == "window":
    #         return [
    #             {"material": "glass_double", "material_uuid": get_uuid("glass_double"), "thickness": 0.04},
    #             {"material": "sealant", "material_uuid": get_uuid("sealant"), "thickness": 0.005},
    #             {"material": "frame_aluminium", "material_uuid": get_uuid("frame_aluminium"), "thickness": 0.005}
    #         ]
    #     elif stype == "door":
    #         return [
    #             {"material": "wood_oak", "material_uuid": get_uuid("wood_oak"), "thickness": 0.035},
    #             {"material": "varnish_finish", "material_uuid": get_uuid("varnish_finish"), "thickness": 0.005}
    #         ]
    #     return []

    def generate_default_layers(stype, thickness):
        if pd.isna(thickness):
            return []

        def get_uuid(material_name):
            return material_lookup.get(material_name.lower())

        if stype in ["external_wall", "internal_wall", "partly_wall"]:
            core_thickness = round(max(thickness - 0.025, 0.05), 5)
            return [
                {"material": "plaster_external", "material_uuid": get_uuid("plaster_external"), "thickness": 0.0125},
                {"material": "bricks_full", "material_uuid": get_uuid("bricks_full"), "thickness": core_thickness},
                {"material": "mortar", "material_uuid": get_uuid("mortar"), "thickness": 0.0125}
            ]
        elif stype == "ceiling":
            return [
                {"material": "plaster_internal", "material_uuid": get_uuid("plaster_internal"), "thickness": 0.015},
                {"material": "concrete_reinforced", "material_uuid": get_uuid("concrete_reinforced"), "thickness": round(0.35 - 0.03, 5)},
                {"material": "plaster_external", "material_uuid": get_uuid("plaster_external"), "thickness": 0.015}
            ]
        elif stype == "floor":
            return [
                {"material": "wood_parquet", "material_uuid": get_uuid("wood_parquet"), "thickness": 0.02},
                {"material": "screed", "material_uuid": get_uuid("screed"), "thickness": 0.05},
                {"material": "concrete_reinforced", "material_uuid": get_uuid("concrete_reinforced"), "thickness": round(0.35 - 0.07, 5)}
            ]
        elif stype == "window":
            return [
                {"material": "glass_double", "material_uuid": get_uuid("glass_double"), "thickness": 0.0125},
                {"material": "sealant", "material_uuid": get_uuid("sealant"), "thickness": 0.085},
                {"material": "frame_aluminium", "material_uuid": get_uuid("frame_aluminium"), "thickness": 0.015}
            ]
        elif stype == "door":
            return [
                {"material": "wood_oak", "material_uuid": get_uuid("wood_oak"), "thickness": 0.035},
                {"material": "varnish_finish", "material_uuid": get_uuid("varnish_finish"), "thickness": 0.005}
            ]
        return []


    material_lookup = {
        str(row["name"]).lower(): row.get("uuid") for _, row in df_materials.iterrows()
    }

    df_thematic_surfaces["surface_type"] = df_thematic_surfaces["thematic_surface"].apply(normalize_surface_type)
    df_thematic_surfaces["centroid"] = df_thematic_surfaces["surface_vertices"].apply(get_centroid)
    df_thematic_surfaces["matched_thickness"] = np.nan

    wall_mask = df_thematic_surfaces["surface_type"].isin(["external_wall", "internal_wall", "partly_wall"])
    df_thematic_surfaces.loc[wall_mask, "matched_thickness"] = df_thematic_surfaces.loc[wall_mask, "centroid"].apply(
        lambda c: find_thickness(c, wall_bboxes)
    )

    df_thematic_surfaces["matched_thickness"] = df_thematic_surfaces.apply(assign_default_thickness_general, axis=1)

    df_thematic_surfaces["thickness_rounded"] = df_thematic_surfaces["matched_thickness"].apply(
        lambda x: round(x / 0.025) * 0.025 if pd.notna(x) else np.nan
    )
    df_thematic_surfaces["core_material"] = df_thematic_surfaces["surface_type"].apply(assign_core_material)
    df_thematic_surfaces["construction_name"] = df_thematic_surfaces.apply(assign_construction_name, axis=1)
    df_thematic_surfaces["normalized_construction_name"] = df_thematic_surfaces["construction_name"].apply(normalize_construction_name)

    # --- Normalize existing constructions ---
    df_constructions["normalized_name"] = df_constructions["name"].apply(normalize_construction_name)

    # --- Add missing constructions ---
    existing_names = set(df_constructions["normalized_name"])
    needed_names = set(df_thematic_surfaces["normalized_construction_name"])
    missing_names = needed_names - existing_names

    missing_rows = df_thematic_surfaces[df_thematic_surfaces["normalized_construction_name"].isin(missing_names)]
    missing_unique = missing_rows[["normalized_construction_name", "surface_type", "thickness_rounded"]].drop_duplicates()

    missing_thickness = df_thematic_surfaces[df_thematic_surfaces["thickness_rounded"].isna()]
    print(f"⚠️ Missing thickness for {len(missing_thickness)} surfaces before fallback.")

    fallback_value = 0.15  

    df_thematic_surfaces["matched_thickness"].fillna(fallback_value, inplace=True)
    df_thematic_surfaces["thickness_rounded"] = df_thematic_surfaces["matched_thickness"].apply(
        lambda x: round(x / 0.025) * 0.025
    )

    missing_after_fallback = df_thematic_surfaces[df_thematic_surfaces["thickness_rounded"].isna()]
    print(f"✅ Missing thicknesses after fallback: {len(missing_after_fallback)}")

    new_constructions = []
    for _, row in missing_unique.iterrows():
        new_constructions.append({
            "name": row["normalized_construction_name"],
            "normalized_name": row["normalized_construction_name"],
            "wall_type": row["surface_type"],
            "match_thickness": row["thickness_rounded"],
            "layers": generate_default_layers(row["surface_type"], row["thickness_rounded"]),
            "construction_uuid": f'construction_uuid_{str(uuid.uuid4())}'
        })

    df_constructions = pd.concat([df_constructions, pd.DataFrame(new_constructions)], ignore_index=True)

    construction_lookup = {
        row["normalized_name"]: row for _, row in df_constructions.iterrows()
    }

    df_thematic_surfaces["construction_uuid"] = df_thematic_surfaces["normalized_construction_name"].map(
        lambda name: construction_lookup.get(name, {}).get("construction_uuid", None)
    )

    df_thematic_surfaces["construction_layers"] = df_thematic_surfaces["normalized_construction_name"].map(
        lambda name: construction_lookup.get(name, {}).get("layers", [])
    )

    df_thematic_surfaces["construction_layers_summary"] = df_thematic_surfaces["construction_layers"].apply(extract_layer_summary)

    missing_final = df_thematic_surfaces[df_thematic_surfaces["construction_uuid"].isna()]
    print(f"✅ Processing complete. Remaining missing constructions: {len(missing_final)}")
    if not missing_final.empty:
        print("Unmatched construction names:")
        print(missing_final["normalized_construction_name"].unique())

    df_thematic_surfaces.drop(columns=["normalized_construction_name", "construction_layers_summary"], inplace=True)
    df_thematic_surfaces.head(50)



⚠️ Missing thickness for 0 surfaces before fallback.
✅ Missing thicknesses after fallback: 0
✅ Processing complete. Remaining missing constructions: 0


C:\Users\oscar\AppData\Local\Temp\ipykernel_29624\3884673718.py:298: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 2000)

df_thematic_surfaces.head(50)

,element_id,parent_id,thematic_surface,has_holes,surface_vertices,surface_type,centroid,matched_thickness,thickness_rounded,core_material,construction_name,construction_uuid,construction_layers
0,2,1.0,partly_wall,False,"[(-39.014, 22.381, 6.657), (-39.119, 29.884, 6.657), (-39.119, 29.884, 3.052), (-39.014, 22.381, 3.052), (-39.014, 22.381, 6.657)]",partly_wall,"[-39.056000000000004, 25.3822, 5.215]",0.259339,0.250,bricks,partly_wall_bricks_25cm,construction_uuid_d07b5e1e-272d-460c-8463-a157d48f78c0,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.225}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]"
1,3,1.0,external,False,"[(-39.119, 29.884, 6.657), (-31.666, 29.983, 6.657), (-31.666, 29.983, 3.052), (-39.119, 29.884, 3.052), (-39.119, 29.884, 6.657)]",external_wall,"[-36.1378, 29.9236, 5.215]",0.644637,0.650,bricks,external_wall_bricks_65cm,construction_uuid_90d45037-c187-4693-9f2c-e16705a08b48,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.625}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]"
2,3,1.0,external_with_hole,True,"[(-31.666, 29.983000000000004, 6.657), (-31.666, 29.983000000000004, 3.052), (-39.119, 29.884000000000004, 3.052), (-39.119, 29.884000000000004, 6.657), (-31.666, 29.983000000000004, 6.657)]",external_wall,"[-34.6472, 29.943400000000004, 5.215]",0.644637,0.650,bricks,external_wall_bricks_65cm,construction_uuid_90d45037-c187-4693-9f2c-e16705a08b48,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.625}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]"
3,5,1.0,external,False,"[(-31.666, 29.983, 3.052), (-31.666, 29.983, 6.657), (-31.513, 22.966, 6.657), (-31.513, 22.966, 3.052), (-31.666, 29.983, 3.052)]",external_wall,"[-31.6048, 27.1762, 4.494]",0.343848,0.350,bricks,external_wall_bricks_35cm,construction_uuid_f6a7e6d6-3662-485e-a118-65c57cf9eee0,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.325}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]"
4,6,1.0,external,False,"[(-31.513, 22.966, 3.052), (-31.513, 22.966, 6.657), (-31.502, 22.488, 6.657), (-31.502, 22.488, 3.052), (-31.513, 22.966, 3.052)]",external_wall,"[-31.5086, 22.7748, 4.494]",0.350000,0.350,bricks,external_wall_bricks_35cm,construction_uuid_f6a7e6d6-3662-485e-a118-65c57cf9eee0,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.325}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]"
5,7,1.0,partly_wall,False,"[(-31.502, 22.488, 6.657), (-39.014, 22.381, 6.657), (-39.014, 22.381, 3.052), (-31.502, 22.488, 3.052), (-31.502, 22.488, 6.657)]",partly_wall,"[-34.506800000000005, 22.4452, 5.215]",0.281342,0.275,bricks,partly_wall_bricks_28cm,construction_uuid_8ffd080e-2672-4103-b26a-7a26db2c9c4e,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.25}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-

CEILINGS AND FLOORS

In [ ]:
# --- STEP 1: Create lambda lookup ---
material_lambda_lookup = {
    mat["name"].lower(): mat.get("lambda_W_per_mK", np.nan)
    for mat in materials_library
}

# --- STEP 2: Check for missing lambda values ---
missing_materials = set()
for layers in df_constructions["layers"]:
    try:
        parsed_layers = literal_eval(layers) if isinstance(layers, str) else layers
        for layer in parsed_layers:
            mat_name = layer.get("material", "").lower()
            if mat_name and material_lambda_lookup.get(mat_name) in [None, np.nan]:
                missing_materials.add(mat_name)
    except Exception:
        continue

if missing_materials:
    print("⚠️ Materials missing lambda values:")
    print(sorted(missing_materials))
else:
    print("✅ All materials used in constructions have lambda values.")

# --- STEP 3: Compute U-values ---
def compute_u_value_from_layers(layers):
    try:
        layers = literal_eval(layers) if isinstance(layers, str) else layers
        if not isinstance(layers, list):
            return np.nan

        r_total = 0.0
        for layer in layers:
            material = layer.get("material", "").lower()
            thickness = layer.get("thickness", 0)
            lam = material_lambda_lookup.get(material)
            if thickness and lam and lam > 0:
                r_total += thickness / lam

        return round(1.0 / r_total, 3) if r_total > 0 else np.nan
    except Exception:
        return np.nan

df_constructions["u_value"] = df_constructions["layers"].apply(compute_u_value_from_layers)

# --- STEP 4: Assign to thematic surfaces ---
construction_u_lookup = df_constructions.set_index("construction_uuid")["u_value"].to_dict()
df_thematic_surfaces["u_value"] = df_thematic_surfaces["construction_uuid"].map(construction_u_lookup)

# --- STEP 5: Fallback for ceiling, floor, window, door ---
standard_u_values = {
    "ceiling": 0.45,
    "floor": 0.40,
    "window": 2.4,
    "door": 1.8
}

# Ensure surface_type still exists
if "surface_type" in df_thematic_surfaces.columns:
    for stype, uval in standard_u_values.items():
        mask = (df_thematic_surfaces["surface_type"] == stype) & (df_thematic_surfaces["u_value"].isna())
        df_thematic_surfaces.loc[mask, "u_value"] = uval

# --- STEP 6: Verify ---
assert "u_value" in df_thematic_surfaces.columns, "❌ u_value column missing!"
assert df_thematic_surfaces["u_value"].notna().any(), "❌ No u-values assigned!"

# --- STEP 7: Final cleanup ---
df_thematic_surfaces.drop(columns=["core_material", "centroid", "surface_type", "surface_vertices"], errors="ignore", inplace=True)

# --- STEP 8: Display result ---
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 2000)

df_thematic_surfaces.head(50)



✅ All materials used in constructions have lambda values.


,element_id,parent_id,thematic_surface,has_holes,matched_thickness,thickness_rounded,construction_name,construction_uuid,construction_layers,u_value
0,2,1.0,partly_wall,False,0.259339,0.250,partly_wall_bricks_25cm,construction_uuid_d07b5e1e-272d-460c-8463-a157d48f78c0,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.225}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",2.939
1,3,1.0,external,False,0.644637,0.650,external_wall_bricks_65cm,construction_uuid_90d45037-c187-4693-9f2c-e16705a08b48,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.625}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",1.116
2,3,1.0,external_with_hole,True,0.644637,0.650,external_wall_bricks_65cm,construction_uuid_90d45037-c187-4693-9f2c-e16705a08b48,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.625}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",1.116
3,5,1.0,external,False,0.343848,0.350,external_wall_bricks_35cm,construction_uuid_f6a7e6d6-3662-485e-a118-65c57cf9eee0,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.325}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",2.087
4,6,1.0,external,False,0.350000,0.350,external_wall_bricks_35cm,construction_uuid_f6a7e6d6-3662-485e-a118-65c57cf9eee0,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.325}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",2.087
5,7,1.0,partly_wall,False,0.281342,0.275,partly_wall_bricks_28cm,construction_uuid_8ffd080e-2672-4103-b26a-7a26db2c9c4e,"[{'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.0125}, {'material': 'bricks_full', 'material_uuid': 'b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002', 'thickness': 0.25}, {'material': 'mortar', 'material_uuid': 'b0a6a2d1-2002-4ff6-a7f9-1b98c7b1f012', 'thickness': 0.0125}]",2.666
6,10,1.0,ceiling,False,0.250000,0.250,ceiling_concrete_25cm,construction_uuid_ef483466-3798-4fcf-b3f7-3bbdbc566b51,"[{'material': 'plaster_internal', 'material_uuid': 'b0a6a2d1-1001-4ff6-a7f9-1b98c7b1f001', 'thickness': 0.015}, {'material': 'concrete_reinforced', 'material_uuid': 'b0a6a2d1-1004-4ff6-a7f9-1b98c7b1f004', 'thickness': 0.32}, {'material': 'plaster_external', 'material_uuid': 'b0a6a2d1-1010-4ff6-a7f9-1b98c7b1f010', 'thickness': 0.015}]",5.265
7,9,1.0,floor,False,0.300000,0.300,floor_concrete_30cm,construction_uuid_8578934a-08c0-4dad-a902-a4b84aa96572,"[{'material': 'wood_parquet', 'material_uuid': 'b0a6a2d1-2004-4ff6-a7f9-1b98c7b1f014', 'thickness': 0.02}, {'material': 'screed', 'material_uuid': 'b0a6a2d1-2003-4ff6-a7f9-1b98c7b1f013', 'thickness': 0.05}, {'material': 'concrete_reinforced', 'material_uuid': 'b0a6a2d1-1004-4ff6-a7f9-1b98c7b1f004', 'thickness': 0.28}]",3.228
8,4,3.0,window,True,0.050000,0.050,window_glass_5cm,construction_uuid_25e90137-ca00-4146-93a1-af131db29ce4,"[{'material': 'glass_double', 'material_uuid': 'b0a6a2d1-2006-4ff6-a7f9-1b98c7b1f016', 'thickness': 0.0125}, {'material': 'sealant', 'material_uuid'

In [ ]:
material_constructions = os.path.join(current_directory, 'outputs', 'materials', f'{name}_material_constructions.csv')                                   
df_thematic_surfaces.to_csv(material_constructions)
